# News Parsing Version 1

## Рекомендуемая структура проекта

In [ ]:
news_parser/
│
├── .env                    # Секретные переменные (в .gitignore)
├── .gitignore              # Игнорируемые файлы
├── requirements.txt        # Зависимости
├── pyproject.toml          # Альтернатива requirements.txt (опционально)
│
├── config/                 
│   ├── __init__.py         # Пустой файл для импортов
│   ├── settings.py         # Общие настройки (несекретные)
│   ├── telegram.py         # Настройки бота
│   └── sites/              # Конфиги для разных сайтов
│       ├── vc_ru.json      
│       └── rbc.json        
│
├── data/
│   ├── raw/                # Сырые данные (кеш HTML)
│   │   ├── vc_ru/         
│   │   └── rbc/           
│   │
│   ├── interim/            # Промежуточные данные
│   │   └── dynamic_page.html  
│   │
│   ├── processed/          # Обработанные данные
│   │   ├── articles.csv    
│   │   └── articles.json   
│   │
│   ├── encrypted/          # Зашифрованные данные (если нужно)
│   ├── backups/            # Автоматические бэкапы
│   └── state/              # Состояния для возобновления
│       └── last_failure.pkl  
│
├── logs/                   # Логи работы
│   ├── parsing_20240515.log  
│   └── errors.log          
│
├── parsers/                # Модули парсинга
│   ├── __init__.py         
│   ├── base_parser.py      # Базовый класс
│   ├── vc_ru.py           # Парсер для VC.ru
│   ├── rbc.py             # Парсер для RBC
│   └── utils/             
│       ├── anti_block.py   # Обход блокировок
│       └── helpers.py      # Вспомогательные функции
│
├── storage/                # Работа с хранилищами
│   ├── __init__.py         
│   ├── local.py           # CSV/JSON
│   ├── google_sheets.py   
│   └── database.py        # Для будущей БД
│
├── notifications/          # Уведомления
│   ├── __init__.py         
│   ├── telegram.py        
│   └── email.py           # Резервный канал
│
├── main.py                 # Точка входа
└── run.bat                 # Скрипт запуска (Windows)

### Ключевые особенности

#### 1. Безопасность данных:
- Все секреты (токены, API-ключи) хранятся в .env (добавлен в .gitignore)
- Пример .env:

In [ ]:
# ini
TELEGRAM_TOKEN=your_telegram_token
GSHEETS_CREDENTIALS=credentials.json
DB_PASSWORD=secret

#### 2. Резервное копирование:
- Автоматические бэкапы в data/backups/ с датой в имени файла:

In [ ]:
# python
shutil.copy2("data/processed/articles.csv", f"data/backups/articles_{datetime.now().date()}.csv")

#### 3. Шифрование (опционально):
- Для чувствительных данных используйте data/encrypted/ с библиотекой cryptography.

#### 4. Логирование:
- Настройка в main.py:

In [ ]:
# python
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logs/parsing.log"),
        logging.StreamHandler()
    ]
)

#### 5. Пример .gitignore

In [ ]:
# gitignore
# Секреты
.env
credentials.json
data/encrypted/*

# Виртуальное окружение
venv/
.env/

# Кеши и временные файлы
__pycache__/
*.py[cod]
data/raw/
data/interim/
data/state/

# IDE
.idea/
.vscode/

#### 6. Как начать использовать
- 1. Создайте структуру папок:

In [ ]:
# bash
mkdir -p news_parser/{config/sites,data/{raw,interim,processed,backups,state},logs,parsers/utils,storage,notifications}

# Чтобы создать все эти папки нужно выйти в родительский каталог (или папка будет создана внутри: news_parser/news_parser)

In [ ]:
# - 2. Установите зависимости:
# bash
pip install -r requirements.txt

In [ ]:
# - 3. Заполните .env по образцу .env.example (создайте его для примера).

#### 7. Следующие шаги
Когда будете готовы продолжить, мы можем:

- Реализовать систему плавного возобновления после сбоев.
- Добавить мониторинг изменений на сайтах (алерт при изменении структуры).
- Настроить автоматические бэкапы в облако (S3, Yandex Disk).

## 0. Виртуальное окружение (обязательно!)

### Перед созданием файлов нужно изолировать зависимости проекта с помощью venv или conda:

In [ ]:
# bash
python -m venv venv  # Создаем виртуальное окружение
source venv/bin/activate  # Активируем (Linux/Mac)
venv\Scripts\activate     # Активируем (Windows)

### 2. Файл requirements.txt
Все библиотеки из вашего кода нужно сохранить в файле requirements.txt:

In [ ]:
# text
# requirements.txt
beautifulsoup4==4.12.0
requests==2.31.0
selenium==4.15.0
webdriver-manager==4.0.0
pandas==2.1.0
gspread==5.11.0
telebot==0.0.5
python-dotenv==1.0.0  # Для загрузки переменных окружения (например, токенов)

In [ ]:
# Команда для установки:
# bash
pip install -r requirements.txt

### 3. Альтернатива: pyproject.toml (для современных проектов)
Если проект сложный, можно использовать poetry или pipenv. Пример для pyproject.toml:

In [ ]:
# toml
[project]
dependencies = [
    "beautifulsoup4>=4.12.0",
    "selenium>=4.15.0",
    "gspread>=5.11.0",
]

### 4. Отдельный файл для констант/настроек
Библиотеки можно разделить на логические группы в файле config.py или constants.py:

In [ ]:
# python
# config.py
# Основные
import os
import json
from datetime import datetime

# Парсинг
from bs4 import BeautifulSoup
import requests

# Динамические сайты
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Google Sheets
import gspread

### 5. Где хранить токены/ключи?
Никогда не кладите их прямо в код! Используйте:
Файл .env (через библиотеку python-dotenv):

In [ ]:
# text
# .env
TELEGRAM_BOT_TOKEN=your_token_here
GSHEET_API_KEY=your_key_here

#### Или настройки ОС (через os.environ).

### 6. Рекомендации:
#### 1. gitignore:
Добавьте исключения для виртуального окружения и секретных файлов:

In [ ]:
# text
venv/
.env
*.bat
__pycache__/

#### 2. Автоматизация:
Для .bat-файла (Windows) можно добавить команды активации окружения и запуска:

In [ ]:
# bat
@echo off
call venv\Scripts\activate
python main.py
pause

### 1. Проверка существующих ключей
Вы уже выполнили:

In [ ]:
# bash
ls -al ~/.ssh

#### Типичные названия ключей:
id_rsa и id_rsa.pub (RSA)
id_ed25519 и id_ed25519.pub (Ed25519)

###  2. Запуск ssh-agent
#### Для Windows (Git Bash):
bash
eval $(ssh-agent -s)  # Запуск агента
ssh-add ~/.ssh/id_rsa  # Добавление ключа (замените id_rsa на ваш ключ)

#### Для Linux/Mac:
bash
eval "$(ssh-agent -s)"
ssh-add ~/.ssh/id_rsa

### 3. Если ключ защищён паролем
Агент запросит пароль от ключа. Введите его.

### 4. Проверка добавления ключа
bash
ssh-add -l  # Список добавленных ключей

### 5. Добавление ключа в GitHub
#### 1. Скопируйте публичный ключ (*.pub):

In [ ]:
# bash
cat ~/.ssh/id_rsa.pub | clip  # Windows (Git Bash)
cat ~/.ssh/id_rsa.pub | pbcopy  # Mac
cat ~/.ssh/id_rsa.pub  # Linux (скопируйте вручную)

#### 2. В GitHub:
Settings → SSH and GPG keys → New SSH key.
Вставьте ключ и сохраните.

### 6. Проверка подключения

In [ ]:
# bash
ssh -T git@github.com

#### Ожидаемый ответ:
Hi username! You've successfully authenticated, but GitHub does not provide shell access. 

In [ ]:
# 


### 7. Настройка Git на использование SSH
Замените ссылку в remote:

bash
git remote set-url origin git@github.com:ваш-логин/news_parser.git
Или при первом подключении:

bash
git remote add origin git@github.com:ваш-логин/news_parser.git

### Частые проблемы и решения
#### 1. Ошибка «Could not open a connection to your authentication agent»
Выполните перед ssh-add:

In [ ]:
# bash
eval $(ssh-agent -s)

In [ ]:
#### 2. Ключ не добавляется
Убедитесь, что ключ в папке ~/.ssh/.
Проверьте права на файлы:

In [ ]:
# bash
chmod 600 ~/.ssh/id_rsa
chmod 644 ~/.ssh/id_rsa.pub

### 3. GitHub не принимает ключ
Скопируйте ключ полностью, включая ssh-ed25519 AAA... или ssh-rsa AAA....
Убедитесь, что не добавляете приватный ключ (id_rsa вместо id_rsa.pub).

### Итоговая последовательность команд

In [ ]:
# bash
# 1. Запуск агента
eval $(ssh-agent -s)

# 2. Добавление ключа
ssh-add ~/.ssh/id_rsa

# 3. Проверка
ssh-add -l
ssh -T git@github.com

# 4. Настройка Git
git remote set-url origin git@github.com:ваш-логин/news_parser.git

##### После этого git push/pull будет работать без пароля! 🔑

## Логирование

### 1. Логирование vs print()

Главные отличия:
Характеристика	    print()	                        Логирование (logging)
Назначение	        Вывод в консоль (для человека)	Запись в файлы/системы (для анализа)
Гибкость	        Только консоль	                Файлы, базы данных, email, облако и т.д.
Уровни сообщений	Нет (всё выводится)	            DEBUG, INFO, WARNING, ERROR
Производительность	Медленнее (блокирующий вывод)	Быстрее (асинхронная запись)
Долговременность	Исчезает после закрытия консоли	Хранится в файлах месяцами

### 2. Пример из dynamic_parse()
Исходный код с логированием:

In [ ]:
# python
import logging

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/parser.log'),  # Запись в файл
        logging.StreamHandler()                 # Вывод в консоль
    ]
)
logger = logging.getLogger(__name__)

def dynamic_parse(url: str, max_scrolls: int) -> Optional[str]:
    try:
        logger.info(f"Начало парсинга: {url}")
        driver = webdriver.Chrome()
        driver.get(url)
        
        for _ in range(max_scrolls):
            logger.debug(f"Прокрутка #{_}")  # Только для отладки
            human_like_interaction(driver)
            
        logger.info(f"Успешно завершено. Загружено страниц: {max_scrolls}")
        return driver.page_source
        
    except Exception as e:
        logger.error(f"Ошибка парсинга: {e}", exc_info=True)
        return None

### 3. Как пользоваться результатами на практике?
#### 3.1. Где искать логи?
Файлы: В папке logs/parser.log (пример содержимого):

In [ ]:
# log
2024-05-20 12:30:45 - __main__ - INFO - Начало парсинга: https://vc.ru
2024-05-20 12:30:50 - __main__ - ERROR - Ошибка парсинга: TimeoutException...
Traceback (most recent call last):
  File "...", line 15, in dynamic_parse
    driver.get(url)
selenium.common.exceptions.TimeoutException: ...

# Консоль: Аналогичный вывод (если настроен StreamHandler).

#### 3.2. Уровни логирования и их использование

Уровень	Когда использовать?	                Пример
DEBUG	Детали для отладки	                logger.debug(f"Прокрутка #{_}")
INFO	Ключевые этапы работы	            logger.info("Парсинг начат")
WARNING	Не критичные, но странные события	logger.warning("Нет данных")
ERROR	Ошибки, требующие внимания	        logger.error("Сайт недоступен")

#### Как фильтровать:

In [ ]:
# python
logging.basicConfig(level=logging.WARNING)  # Только WARNING и выше

### 3.3. Анализ логов
#### 1. Поиск ошибок:

In [ ]:
# bash
grep "ERROR" logs/parser.log

#### 2. Статистика:

In [ ]:
# bash
awk '/INFO/ {count++} END {print "Всего успешных запусков:", count}' logs/parser.log

#### 3. Мониторинг (например, через journalctl или облачные сервисы типа Datadog).

### 4. Преимущества для вашего проекта
#### 1. Отслеживание сбоев:
Если парсер упадёт ночью, вы увидите ошибку в логах, а не «молчание».

#### 2. Оптимизация:
Логи типа DEBUG помогут найти «узкие» места (например, долгие прокрутки).

#### 3. Анализ истории:
Сравнивайте логи за разные дни, чтобы выявить проблемы на сайте (например, изменилась структура HTML).

### 5. Советы
#### 1. Ротация логов (чтобы файлы не весили гигабайты):

In [ ]:
# python
from logging.handlers import RotatingFileHandler
handler = RotatingFileHandler('logs/parser.log', maxBytes=1_000_000, backupCount=5)

#### 2. Сенситивные данные:
Не логируйте пароли/токены! Фильтруйте их:

In [ ]:
# python
logger.info(f"URL: {url.replace('api_key=XXX', 'api_key=REDACTED')}")

#### Готовые решения:
Для сложных проектов используйте:
loguru (удобнее стандартного logging),
Sentry (для мониторинга ошибок).

### Пример улучшенного лога
После запуска dynamic_parse("https://vc.ru") в файле logs/parser.log появится:

In [ ]:
# log
2024-05-20 12:30:45 - parsers.dynamic - INFO - Начало парсинга: https://vc.ru
2024-05-20 12:30:47 - parsers.dynamic - DEBUG - Прокрутка #1
2024-05-20 12:30:50 - parsers.dynamic - INFO - Успешно завершено. Загружено страниц: 10

#### Теперь вы можете:
Найти все ошибки за месяц,
Измерить среднее время парсинга,
Доказать, что проблема на стороне сайта (если в логах есть Timeout).

#### Логирование — это «чёрный ящик» вашей программы. Настроив его один раз, вы сэкономите часы на отладке! Если нужно адаптировать под конкретные задачи — спрашивайте.

## 1. Загрузка библиотек - START_TEST_1 !!! - START_1 !!!
Все библиотеки для работы проекта в одном месте

In [1]:
# Загрузка библиотек
# Для вспомогательных функций и основного парсера
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urljoin

# Для экспорта/импорта временных данных
import json
import csv

# Для работы с динамическими сайтами
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.remote.webdriver import WebDriver
import time
import random

# Обновление CSV/JSON с дедупликацией
import pandas as pd
from pathlib import Path

# Google Sheets
import gspread
from gspread import Client, Spreadsheet, Worksheet, service_account, exceptions
from typing import Dict, List, Optional, Union
from google.oauth2.service_account import Credentials

# Библиотеки для ТелеБОТа
import telebot
from telebot import types
import traceback
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

# для указание типов
from abc import ABC, abstractmethod
from typing import Dict, List, Optional

# Логирование
import logging

# токен лежит в файле config.py
import config

777. Hello Nick!
888. Notebook testing SCHEDULE
222. Token_&_CHAT_ID - Gut !!


## 2. DYNAMIC_PARSE()
Динамически загружает контент с имитацией поведения человека.

### 0. Первоначальные версии DYNAMIC_PARSE - HUMAN_LIKE_INTERACTION (из файла PARSUNG_WITH_SIMYCH)

#### human_like_interaction()

In [ ]:
def human_like_interaction(driver):
    """Имитирует действия человека: плавная прокрутка, hover, случайные паузы."""
    # Плавная прокрутка (3-5 сек)
    scroll_pause = random.uniform(3, 5)
    scroll_amount = random.randint(800, 1200)
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
    time.sleep(scroll_pause)

    # Случайный hover на элементы (30% вероятность)
    if random.random() > 0.7:
        elements = driver.find_elements(By.CSS_SELECTOR, ".content-title, .author__name, .content-header__item")
        if elements:
            element = random.choice(elements)
            ActionChains(driver).move_to_element(element).pause(1).perform()
            time.sleep(random.uniform(1, 2))

#### dynamic_parse()

In [ ]:
# Динамически загружает контент в локальный html файл
def dynamic_parse(url="https://vc.ru/new", max_scrolls=10):
    """
    Динамически загружает контент с имитацией поведения человека.
    
    Параметры:
        url (str): URL для парсинга
        max_scrolls (int): Максимальное число прокруток
        
    Возвращает:
        str: HTML-контент или None при ошибке
    """
    try:
        # Настройка драйвера
        service = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 115)}.0.0.0 Safari/537.36")
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Счетчик прокрутки
        count = 1

        # Основной цикл прокрутки
        for _ in range(max_scrolls):
            human_like_interaction(driver)
            print(f"\tПрокрутка: {count}")
            count+=1
            # Проверка на кнопку "Показать ещё" (если есть)
            try:
                more_btn = driver.find_element(By.CSS_SELECTOR, ".button-more")
                if more_btn.is_displayed():
                    more_btn.click()
                    time.sleep(random.uniform(2, 4))
            except:
                pass
                
        print(f"\tПрокручено: {count} раз!")
        print("\n✅ 1. HTML сохранён с динамическим контентом!")
        return driver.page_source

    except Exception as e:
        print(f"🚨 Ошибка в dynamic_parse(): {type(e).__name__} - {str(e)}")
        return None
    finally:
        if 'driver' in locals():
            driver.quit()

In [ ]:
# Пример использования
if __name__ == "__main__":
    # html = dynamic_parse()
    html = dynamic_parse(max_scrolls=50)
    if html:
        with open("data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
            f.write(html)
        print("✅ 1. HTML сохранён с динамическим контентом!")
        print("Скрипт отработал:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

### 1. Модификация dynamic_parse() - Version Nr.1

#### Улучшения:
1. Гибкость настроек:
Вынесем параметры Selenium в конфиг (например, user_agent, timeout).

2. Без промежуточного HTML:
Убираем сохранение в файл (как вы и хотели).

3. Логирование:
Заменим print() на модуль logging.

Обновлённый код:

In [ ]:
# Version Nr.1

# from typing import Optional
# import logging
# from selenium.webdriver.remote.webdriver import WebDriver

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def dynamic_parse(
    url: str = "https://vc.ru/new",
    max_scrolls: int = 10,
    driver: Optional[WebDriver] = None
) -> Optional[str]:
    """
    Динамически загружает контент с имитацией поведения человека.
    
    Args:
        url: URL для парсинга.
        max_scrolls: Максимальное число прокруток.
        driver: Опционально, существующий экземпляр WebDriver.
        
    Returns:
        HTML-контент или None при ошибке.
    """
    local_driver = None
    try:
        if not driver:
            service = Service(ChromeDriverManager().install())
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")  # Для продакшена без GUI
            options.add_argument("--disable-blink-features=AutomationControlled")
            options.add_argument(f"user-agent={get_random_user_agent()}")
            local_driver = webdriver.Chrome(service=service, options=options)
            driver = local_driver

        driver.get(url)
        
        for count in range(1, max_scrolls + 1):
            human_like_interaction(driver)
            logger.info(f"Прокрутка #{count} завершена")
            
            # Обработка кнопки "Показать ещё" (если есть)
            load_more_content(driver)

        logger.info("✅ Динамический контент загружен")
        return driver.page_source

    except Exception as e:
        logger.error(f"Ошибка в dynamic_parse: {type(e).__name__} - {e}", exc_info=True)
        return None
    finally:
        if local_driver:  # Закрываем только если создавали здесь
            local_driver.quit()

def get_random_user_agent() -> str:
    """Генерирует случайный User-Agent."""
    chrome_versions = range(90, 116)
    return f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.choice(chrome_versions)}.0.0.0 Safari/537.36"

def load_more_content(driver: WebDriver) -> None:
    """Пытается найти и нажать кнопку 'Показать ещё'."""
    try:
        more_btn = driver.find_element(By.CSS_SELECTOR, ".button-more")
        if more_btn.is_displayed():
            more_btn.click()
            time.sleep(random.uniform(2, 4))
    except Exception:
        pass

### 2. Модификация human_like_interaction() - рабочие функции, но в проекте не используются

#### Новая, улучшенная версия функции:
1. Разделение логики:
Вынесем плавную прокрутку и hover в отдельные функции.

2. Конфигурируемость:
Параметры (паузы, вероятность hover) можно вынести в конфиг.

Обновлённый код:

In [27]:
#  вспомогательные функции - рабочие функции, но в проекте не используются
def human_like_interaction(driver: WebDriver) -> None:
    """Имитирует действия человека."""
    smooth_scroll(driver)
    random_hover(driver)

def smooth_scroll(driver: WebDriver, scroll_amount: Optional[int] = None) -> None:
    """Плавная прокрутка страницы."""
    scroll_pause = random.uniform(3, 5)
    scroll_amount = scroll_amount or random.randint(800, 1200)
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
    time.sleep(scroll_pause)

def random_hover(driver: WebDriver, probability: float = 0.3) -> None:
    """Случайный hover на элементы (по умолчанию 30% вероятность)."""
    if random.random() > probability:
        elements = driver.find_elements(By.CSS_SELECTOR, ".content-title, .author__name, .content-header__item")
        if elements:
            element = random.choice(elements)
            ActionChains(driver).move_to_element(element).pause(1).perform()
            time.sleep(random.uniform(1, 2))

### 3. Модификация dynamic_parse() - Version Nr.2 - рабочие функции, но в проекте не используются

In [28]:
# 
# Version Nr.2
# import logging

# Настройка логирования
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('logs/parser.log'),  # Запись в файл если запуск из MAIN.PY
        logging.FileHandler('../logs/parser.log'),  # Запись в файл если запуск из JUPYTER_NOTEBOOK
        logging.StreamHandler()                 # Вывод в консоль
    ]
)
logger = logging.getLogger(__name__)

def dynamic_parse(url: str, max_scrolls: int) -> Optional[str]:
    try:
        logger.info(f"Начало парсинга: {url}")
        driver = webdriver.Chrome()
        driver.get(url)
        
        for _ in range(max_scrolls):
            logger.debug(f"Прокрутка #{_}")  # Только для отладки
            human_like_interaction(driver)
            
        logger.info(f"Успешно завершено. Загружено страниц: {max_scrolls}")
        return driver.page_source
        
    except Exception as e:
        logger.error(f"Ошибка парсинга: {e}", exc_info=True)
        return None

#### Интеграция в проект - START_TEST_4 !!!

In [ ]:
# Пример использования для парсинга "с лёту" - без сохранения в файл - START_TEST_4.1 !!!:
# from parsers.dynamic import dynamic_parse

html = dynamic_parse(url="https://vc.ru/new", max_scrolls=5)
# if html:
    # Парсим "с лёту" без сохранения в файл
    # soup = BeautifulSoup(html, 'html.parser')
    # ... обработка данных ...

In [29]:
# Пример использования для парсинга с промежуточным сохранением в HTML-файл - START_TEST_4.2 !!!:

html = dynamic_parse(url="https://vc.ru/new", max_scrolls=5)
if html:
    with open("../data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("✅ 1. HTML сохранён с динамическим контентом!")
    print("\tСкрипт отработал:", datetime.now().strftime('%d.%m.%Y %H:%M:%S'))

2025-05-11 21:27:17,808 - __main__ - INFO - Начало парсинга: https://vc.ru/new
2025-05-11 21:30:28,948 - __main__ - ERROR - Ошибка парсинга: HTTPConnectionPool(host='localhost', port=63447): Read timed out. (read timeout=120)
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\ProgramData\anaconda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.f

### 4. Улучшенная функция dynamic_parse() с retry-механизмом - Version Nr.3

##### Ключевые улучшения:
1. Retry-механизм:
- Автоматические повторные попытки при таймаутах
- Прогрессивная задержка между попытками
- Логирование каждой попытки

2. Устойчивость к ошибкам:
- Обработка специфических исключений (WebDriverException, ReadTimeoutError)
- Гарантированное освобождение ресурсов в finally

3. Гибкие параметры:
- Настраиваемое количество попыток
- Настраиваемые таймауты

4. Улучшенное логирование:
- Подробные сообщения о каждой попытке
- Разделение временных и критических ошибок

#### Улучшенная функция human_like_interaction() - START_TEST_2 !!! - START_2 !!!

In [2]:
#  - START_TEST_2 !!! - START_2 !!!
def human_like_interaction(driver, max_attempts=3):
    """Имитация человеческого поведения с обработкой ошибок"""
    for attempt in range(1, max_attempts + 1):
        try:
            # Плавная прокрутка
            scroll_pause = random.uniform(3, 5)
            scroll_amount = random.randint(800, 1200)
            driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
            time.sleep(scroll_pause)
            
            # Случайный hover (30% вероятность)
            if random.random() > 0.7:
                elements = driver.find_elements(By.CSS_SELECTOR, ".content-title, .author__name, .content-header__item")
                if elements:
                    element = random.choice(elements)
                    ActionChains(driver).move_to_element(element).pause(1).perform()
                    time.sleep(random.uniform(1, 2))
            return
            
        except Exception as e:
            logging.warning(f"Ошибка взаимодействия (попытка {attempt}): {e}")
            if attempt == max_attempts:
                raise
            time.sleep(2)

#### Улучшенная функция dynamic_parse() с retry-механизмом - START_TEST_3 !!! - START_3 !!!

In [3]:
#  - START_TEST_3 !!! - START_3 !!!
from selenium.common.exceptions import WebDriverException
from urllib3.exceptions import ReadTimeoutError
import time
import random

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('logs/parser.log'),  # Запись в файл если запуск из MAIN.PY
        logging.FileHandler('../logs/parser.log'),  # Запись в файл если запуск из JUPYTER_NOTEBOOK
        logging.StreamHandler()                 # Вывод в консоль
    ]
)
logger = logging.getLogger(__name__)

def dynamic_parse(url="https://vc.ru/new", max_scrolls=10, max_retries=3, retry_delay=5):
    """
    Динамически загружает контент с автоматическими повторами при ошибках.
    
    Args:
        url: URL для парсинга
        max_scrolls: Максимальное число прокруток
        max_retries: Максимальное количество попыток
        retry_delay: Задержка между попытками (в секундах)
        
    Returns:
        str: HTML-контент или None при ошибке
    """
    service = None
    driver = None
    
    for attempt in range(1, max_retries + 1):
        try:
            # Настройка драйвера
            service = Service(ChromeDriverManager().install())
            options = webdriver.ChromeOptions()
            options.add_argument("--disable-blink-features=AutomationControlled")
            options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 115)}.0.0.0 Safari/537.36")
            
            driver = webdriver.Chrome(service=service, options=options)
            driver.set_page_load_timeout(60)  # Таймаут загрузки страницы
            
            # Основная логика
            driver.get(url)
            for count in range(1, max_scrolls + 1):
                human_like_interaction(driver)
                logging.info(f"Прокрутка #{count}")
                
                # Проверка на кнопку "Показать ещё"
                try:
                    more_btn = driver.find_element(By.CSS_SELECTOR, ".button-more")
                    if more_btn.is_displayed():
                        more_btn.click()
                        time.sleep(random.uniform(2, 4))
                except:
                    pass
                    
            logging.info(f"Динамический контент успешно загружен. Количество прокруток: {count}.")
            return driver.page_source
            
        except (ReadTimeoutError, WebDriverException, TimeoutError) as e:
            logging.warning(f"Попытка {attempt}/{max_retries} не удалась: {type(e).__name__}\n")
            if attempt < max_retries:
                time.sleep(retry_delay * attempt)  # Увеличиваем задержку с каждой попыткой
                continue
            logging.error("Превышено максимальное количество попыток")
            return None
            
        finally:
            if driver:
                driver.quit()
            if service:
                service.stop()

In [4]:
# Пример использования для парсинга с промежуточным сохранением в HTML-файл - START_TEST_4.2 !!!:

html = dynamic_parse(url="https://vc.ru/new", max_scrolls=5)
if html:
    with open("../data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("✅ 1. HTML сохранён с динамическим контентом!")
    print("\tСкрипт отработал:", datetime.now().strftime('%d.%m.%Y %H:%M:%S'))

✅ 1. HTML сохранён с динамическим контентом!
	Скрипт отработал: 12.05.2025 07:55:44


#### Интеграция с start_parsing()

In [ ]:
# 
def start_parsing():
    try:
        # Увеличенные таймауты и попытки
        html = dynamic_parse(
            url="https://vc.ru/new",
            max_scrolls=50,
            max_retries=5,
            retry_delay=10
        )
        
        if html is None:
            logging.error("Не удалось загрузить страницу после нескольких попыток")
            return
            
        # Остальная логика обработки...
        
    except Exception as e:
        logging.error(f"Критическая ошибка в start_parsing: {e}", exc_info=True)

#### Для работы по расписанию можно добавить внешний retry-декоратор:

##### Это решение:
- Автоматически перезапускает парсинг при падении
- Логирует каждую попытку
- Сохраняет все предыдущие улучшения по обработке ошибок
-  быть настроено на разные интервалы перезапуска

In [ ]:
# 
from functools import wraps
import time

def retry_on_failure(max_retries=3, delay=60):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    logging.error(f"Попытка {attempt} не удалась: {e}")
                    if attempt < max_retries:
                        time.sleep(delay)
                        continue
                    raise
        return wrapper
    return decorator

@retry_on_failure(max_retries=3, delay=300)
def scheduled_parsing():
    start_parsing()

### 4. Что дальше?
1. Тестирование:
Проверить работу с разными сайтами.
Добавить обработку капчи (если появится).

2. Оптимизация:
Кеширование драйвера (не создавать его каждый раз).

3. Безопасность:
Проверка url на валидность перед запросом.

## 3. PARSE_ARTICLES
Запуск основной функции парсинга

### 1. ООП-подход: Создание парсеров для каждого сайта
Идея:
Создаем базовый класс BaseParser с общими методами, а для каждого сайта — наследника с конкретной реализацией.

#### 1.1. Базовая структура  - START_TEST_5 !!! - START_4 !!!

In [4]:
#  - START_TEST_5  - START_4 !!!
# from abc import ABC, abstractmethod
# from typing import Dict, List, Optional
# from bs4 import BeautifulSoup

class BaseParser(ABC):
    def __init__(self, host: str):
        self.host = host

    @abstractmethod
    def parse_articles(self, html: str) -> List[Dict]:
        """Основной метод парсинга. Должен быть реализован для каждого сайта."""
        pass

    # Общие вспомогательные методы (можно переопределять)
    def get_text_or_none(self, parent, selector: str) -> Optional[str]:
        element = parent.select_one(selector)
        return element.get_text(strip=True) if element else None

    def get_link(self, parent, selector: str) -> Optional[str]:
        element = parent.select_one(selector)
        return urljoin(self.host, element['href']) if element and 'href' in element.attrs else None

#### 1.2. Пример для vc.ru - START_TEST_6 !!! - START_5 !!!

In [5]:
#  - START_TEST_6 !!! - START_5 !!!
class VcRuParser(BaseParser):
    def parse_articles(self, html: str) -> List[Dict]:
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.find('div', class_='content-list')
        if not articles:
            raise ValueError("Секция с новостями не найдена")

        results = []
        for article in articles.find_all('div', class_='content--short'):
            # Ищем тег <time> внутри статьи
            time_tag = article.find('time')
            
            # Извлекаем дату из атрибута datetime (формат ISO 8601)
            if time_tag and time_tag.has_attr('datetime'):
                date_iso = time_tag['datetime']  # "2025-04-12T09:43:34.000Z"
                # Конвертируем в нужный формат (опционально)
                date_formatted = datetime.fromisoformat(date_iso.replace('Z', '+00:00')).strftime('%d.%m.%Y %H:%M:%S')
            else:
                date_iso = None
                date_formatted = None
            
            data = {
                'title': self.get_text_or_none(article, 'div.content-title'),
                'author': self.get_text_or_none(article, 'a.author__name'),
                'date': date_formatted,  # Формат "12.04.2025 в 12:43"
                'shorts': self.get_text_or_none(article, 'div.block-text'),
                'link': self.get_link(article, 'a.content__link'),
                'category': self._extract_category(article),
                'parse_datetime': datetime.now().strftime('%d.%m.%Y %H:%M:%S')
            }
            results.append(data)
        
        print("✅ 2. Новые данные сохранёны в переменной ARTICLES для обработки!")
        print("\tСкрипт отработал:", datetime.now().strftime('%d.%m.%Y %H:%M:%S'))
        return results

    def _extract_category(self, article) -> Optional[str]:
        """Уникальный метод для vc.ru."""
        url = self.get_link(article, 'a.content__link')
        if not url:
            return None
        parts = url.replace(self.host, '').strip('/').split('/')
        return parts[0] if parts else None

#### 1.3. Использование - START_TEST_7 !!!

In [ ]:
# Запуск сразу двух функций без сохранения HTML кода (DYNAMIC_PARSE() и PARSE_ARTICLES() должны быть инициализированы)
parser = VcRuParser(host="https://vc.ru")
html = dynamic_parse(url="https://vc.ru/new")
articles = parser.parse_articles(html)

In [8]:
#  - START_TEST_7 !!!
parser = VcRuParser(host="https://vc.ru")
# html = dynamic_parse(url="https://vc.ru/new")
articles = parser.parse_articles(html)

✅ 2. Новые данные сохранёны в переменной ARTICLES для обработки!
	Скрипт отработал: 12.05.2025 07:56:32


### 2. Конфигурационный подход (без ООП)
Идея:
Храним CSS-селекторы для каждого сайта в конфиге (JSON/YAML), а парсер использует их динамически.

#### 2.1. Конфиг (config/sites/vc.ru.json):

In [ ]:
# json
{
  "host": "https://vc.ru",
  "selectors": {
    "articles_container": "div.content-list",
    "article": "div.content--short",
    "fields": {
      "title": "div.content-title",
      "author": "a.author__name",
      "link": "a.content__link"
    }
  },
  "custom_functions": {
    "category": "extract_vc_ru_category"
  }
}

#### 2.2. Универсальный парсер

In [ ]:
# 
def parse_with_config(html: str, config: Dict) -> List[Dict]:
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select_one(config['selectors']['articles_container'])
    results = []

    for article in articles.select(config['selectors']['article']):
        data = {}
        for field, selector in config['selectors']['fields'].items():
            data[field] = get_text_or_none(article, selector)
        
        # Кастомная логика (например, категория)
        if 'custom_functions' in config:
            for field, func_name in config['custom_functions'].items():
                data[field] = globals()[func_name](article)
        results.append(data)
    return results

### 3. Гибридный подход (ООП + Конфиги)
Идея:
Комбинируем оба метода. Базовый класс парсера загружает конфиг, но позволяет переопределять сложную логику.

In [ ]:
# 
class ConfigurableParser(BaseParser):
    def __init__(self, config_path: str):
        with open(config_path) as f:
            self.config = json.load(f)
        super().__init__(self.config['host'])

    def parse_articles(self, html: str) -> List[Dict]:
        # Базовая логика из конфига
        data = parse_with_config(html, self.config)
        # Дополнительная обработка
        return self.post_process(data)

    def post_process(self, data: List[Dict]) -> List[Dict]:
        """Можно переопределить в дочерних классах."""
        return data

### 4. Обработка капчи и защит
Для обхода защит:
##### 1. Rotate User-Agents и прокси:
Используйте библиотеку fake-useragent и сервисы типа ScraperAPI.

##### 2. Задержки между запросами:

In [ ]:
# Задержки между запросами:
time.sleep(random.uniform(1, 3))

##### 3. Headless-браузер:
В Selenium используйте --headless, но добавляйте человекообразные действия (как у вас в human_like_interaction).

### Итог
- Для 1-3 сайтов — ООП-подход (классы-парсеры).
- Для 5+ сайтов — конфиги + возможность кастомной логики.
- Сложные сайты (капча, JS) — Selenium + обходные методы.

## 4. UPDATE_DATA

In [ ]:
Обновление CSV/JSON с дедупликацией (с доработкой для работы с форматами времени)

#### Что изменилось:
Добавлен .assign(parse_datetime=lambda x: x["first_seen"]) — подменяем дату на оригинальную.

### 1. Улучшенная версия update_data() - START_TEST_8 !!! - START_6 !!!

In [6]:
#  - START_TEST_8 !!! - START_6 !!!
# import pandas as pd
from pathlib import Path
# from typing import List, Dict, Optional
# import logging
# from datetime import datetime

# Настройка логирования
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('logs/parser.log'),  # Запись в файл если запуск из MAIN.PY
        logging.FileHandler('../logs/parser.log'),  # Запись в файл если запуск из JUPYTER_NOTEBOOK
        logging.StreamHandler()                 # Вывод в консоль
    ]
)
logger = logging.getLogger(__name__)

def update_data(
    new_data: List[Dict],
    # csv_path: str = "data/output/articles.csv",
    # json_path: str = "data/output/articles.json",
    csv_path: str = "../data/output/articles.csv",
    json_path: str = "../data/output/articles.json",
    date_format: str = "%d.%m.%Y %H:%M:%S",
    dedup_key: str = "title"
) -> Optional[pd.DataFrame]:
    """
    Обновляет данные с дедупликацией, сохраняя первую дату появления записи.
    
    Args:
        new_data: Список новых данных в виде словарей.
        csv_path: Путь к CSV-файлу.
        json_path: Путь к JSON-файлу.
        date_format: Формат даты для парсинга.
        dedup_key: Поле для дедупликации (по умолчанию 'title').
        
    Returns:
        DataFrame с обновлёнными данными или None при ошибке.
    """
    try:
        # Создаём директории, если их нет
        Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
        
        # Загрузка старых данных
        old_df = load_existing_data(csv_path, date_format)
        
        # Обработка новых данных
        new_df = pd.DataFrame(new_data)
        new_df["parse_datetime"] = pd.to_datetime(
            new_df["parse_datetime"], 
            format=date_format,
            dayfirst=True
        )
        
        # Объединение и дедупликация
        combined_df = deduplicate_data(old_df, new_df, dedup_key)
        combined_df["parse_datetime"] = combined_df["parse_datetime"].dt.strftime(date_format)
        count_combined_df = len(combined_df)
        count_new_df = len(combined_df)-len(old_df)
        
        # Сохранение
        save_data(combined_df, csv_path, json_path)
        logger.info(f"Данные обновлены. Уникальных записей: {count_combined_df}. Новых записей: {count_new_df}")
        print("✅ 3. CSV/JSON файлы сохранёны и дополнены новым контентом!")
        print("\tСкрипт отработал:", datetime.now().strftime('%d.%m.%Y %H:%M:%S'))
        return combined_df
        
    except Exception as e:
        logger.error(f"Ошибка в update_data: {e}", exc_info=True)
        return None

def load_existing_data(path: str, date_format: str) -> pd.DataFrame:
    """Загружает существующие данные из CSV."""
    try:
        df = pd.read_csv(path)
        df["parse_datetime"] = pd.to_datetime(
            df["parse_datetime"], 
            format=date_format,
            dayfirst=True
        )
        return df
    except FileNotFoundError:
        return pd.DataFrame()

def deduplicate_data(
    old_df: pd.DataFrame, 
    new_df: pd.DataFrame, 
    key: str
) -> pd.DataFrame:
    """
    Объединяет данные, сохраняя ПЕРВУЮ дату появления записи.
    
    Args:
        old_df: Старые данные из CSV.
        new_df: Новые данные из парсинга.
        key: Поле для дедупликации (например, 'title').
        
    Returns:
        DataFrame без дубликатов с оригинальными датами.
    """
    # Объединяем данные
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
    
    # Сохраняем минимальную дату для каждой записи
    combined_df["first_seen"] = combined_df.groupby(key)["parse_datetime"].transform("min")
    
    # Оставляем последнюю версию записи, но с оригинальной датой
    result_df = (
        combined_df
        .sort_values("parse_datetime", ascending=False)
        .drop_duplicates(key, keep="first")
        .assign(parse_datetime=lambda x: x["first_seen"])  # Возвращаем первую дату
        .drop(columns="first_seen")
    )
    
    return result_df

def save_data(
    df: pd.DataFrame, 
    csv_path: str, 
    json_path: str
) -> None:
    """Сохраняет данные в CSV и JSON."""
    df.to_csv(csv_path, index=False, encoding='utf-8')
    df.to_json(json_path, orient="records", force_ascii=False, indent=2)

In [10]:
# Запуск функции UPDATE_DATA() - START_TEST_9 !!!
if __name__ == "__main__":
    update_data(articles)

✅ 3. CSV/JSON файлы сохранёны и дополнены новым контентом!
	Скрипт отработал: 12.05.2025 07:57:09


In [11]:
# Проверка работы функции deduplicate_data()

old = pd.DataFrame([{"title": "A", "parse_datetime": "2025-01-01"}])
new = pd.DataFrame([{"title": "A", "parse_datetime": "2025-05-10"}])
result = deduplicate_data(old, new, "title")
print(result["parse_datetime"].iloc[0])  # Выведет 2025-01-01

2025-01-01


### 2. Подготовка к интеграции с БД

#### 2.1. Адаптер для БД
Добавим абстрактный класс для работы с разными хранилищами:

In [ ]:
# 
from abc import ABC, abstractmethod

class DataStorage(ABC):
    @abstractmethod
    def save(self, data: pd.DataFrame) -> bool:
        pass
    
    @abstractmethod
    def load(self) -> pd.DataFrame:
        pass

# Пример реализации для PostgreSQL
class PostgresStorage(DataStorage):
    def __init__(self, connection_string: str):
        self.conn = psycopg2.connect(connection_string)
        
    def save(self, data: pd.DataFrame) -> bool:
        try:
            data.to_sql("articles", self.conn, if_exists="replace", index=False)
            return True
        except Exception as e:
            logger.error(f"Ошибка сохранения в PostgreSQL: {e}")
            return False

# Пример для Git (версионирование данных)
class GitDataStorage:
    def __init__(self, repo_path: str):
        self.repo = git.Repo(repo_path)
        
    def commit_data(self, message: str):
        self.repo.git.add("data/")
        self.repo.index.commit(message)

### 3. Защита данных

#### 3.1. Шифрование чувствительных полей

In [ ]:
# 
from cryptography.fernet import Fernet

class DataEncryptor:
    def __init__(self, key: str):
        self.cipher = Fernet(key.encode())
        
    def encrypt_field(self, value: str) -> str:
        return self.cipher.encrypt(value.encode()).decode()
        
    def decrypt_field(self, value: str) -> str:
        return self.cipher.decrypt(value.encode()).decode()

In [ ]:
# 


#### 3.2. Пример использования

In [ ]:
# 
encryptor = DataEncryptor(os.getenv("ENCRYPTION_KEY"))
df["author"] = df["author"].apply(encryptor.encrypt_field)

### 4. Архитектура хранения

#### 4.1. Варианты размещения данных
Способ              Плюсы	                      Минусы
Локальные CSV/JSON	Простота, не требует сервера  Нет версионности, сложный поиск
PostgreSQL	        Поддержка сложных запросов	  Требует сервер
Git-репозиторий	    История изменений, бэкапы	  Медленная работа с большими файлами
Google Sheets	    Доступ из любого места	      Ограничения API

### 5. Интеграция с планировщиком

In [ ]:
# Пример .bat-файла для автоматического обновления:
# bat
@echo off
call venv\Scripts\activate
python -c "from main import update_all; update_all()"
git add data/processed/
git commit -m "Auto-update: %date% %time%"
git push

### 6. Дальнейшие шаги

1. Выбор хранилища:
Для старта хватит CSV + Git.
При росте данных — PostgreSQL.

2. Резервное копирование:
Автоматические бэкапы в облако (S3, Dropbox).

3. Мониторинг:
Логирование всех операций в logs/app.log.

## 5. Функция EXPORT_TO_GOOGLE_SHEETS
Код для "умного" обновления Google Sheets

In [47]:
# Функция для восстановления данных и тестирования - сохраняет НОВЫЕ ДАТЫ ПАРСИНГА !!!!

def merge_and_deduplicate(old: List[Dict], new: List[Dict], key: str) -> List[Dict]:
    """Объединяет данные, сохраняя последнюю версию дубликатов."""
    combined = {item[key]: item for item in old}
    combined.update({item[key]: item for item in new})
    return list(combined.values())

### Вспомогательные функции --- 2 - START_TEST_10 !!! - START_7 !!!

In [7]:
# --- Вспомогательные функции --- 2 - START_TEST_10 !!! - START_7 !!!
def infer_sheet_name(item: Dict) -> str:
    """Определяет имя листа на основе данных."""
    return item.get("category", "General").replace("/", "_")[:30]

def get_or_create_worksheet(spreadsheet, sheet_name: str):
    """Находит или создаёт лист в Google Sheets."""
    try:
        return spreadsheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        return spreadsheet.add_worksheet(title=sheet_name, rows=100, cols=20)

def merge_and_deduplicate(old: List[Dict], new: List[Dict], key: str) -> List[Dict]:
    """
    Объединяет данные, сохраняя ПЕРВУЮ дату появления записи.
    
    Args:
        old: Старые данные из Google Sheets.
        new: Новые данные из парсинга.
        key: Поле для дедупликации (например, 'title').

        Сначала обрабатываются старые данные, чтобы их даты имели приоритет.
        При обновлении записей сохраняется оригинальная дата из old.
        
    Returns:
        Список словарей без дубликатов с оригинальными датами.
    """
    # Собираем все записи в словарь {ключ: данные}
    combined = {}
    
    # Сначала добавляем старые данные (чтобы их даты были приоритетными)
    for item in old:
        if item[key] not in combined:
            combined[item[key]] = item
    
    # Затем обновляем новыми данными, но сохраняем старые даты
    for item in new:
        if item[key] in combined:
            # Сохраняем старую дату из combined
            old_date = combined[item[key]].get("parse_datetime")
            # Обновляем запись, но оставляем старую дату
            combined[item[key]].update(item)
            combined[item[key]]["parse_datetime"] = old_date
        else:
            combined[item[key]] = item
    
    return list(combined.values())

def format_dates(data: List[Dict], time_format: str) -> None:
    """
    Приводит все даты в данных к единому формату.
    
    Args:
        data: Список словарей с данными
        time_format: Целевой формат (например, "%d.%m.%Y %H:%M")
    """
    for item in data:
        if 'date' in item and item['parse_datetime']:
            try:
                # Пробуем распарсить дату из разных форматов
                if isinstance(item['parse_datetime'], str):
                    parsed_date = datetime.strptime(item['date'], "%d.%m.%Y %H:%M:%S")
                else:
                    parsed_date = item['parse_datetime']
                item['parse_datetime'] = parsed_date.strftime(time_format)
            except (ValueError, TypeError):
                item['parse_datetime'] = "N/A"  # Если дата некорректна

def update_worksheet(worksheet, data: List[Dict], batch_size: int) -> None:
    """Обновляет лист пачками (совместимость с gspread >=4.0)."""
    headers = list(data[0].keys())
    values = [headers] + [list(item.values()) for item in data]
    
    for i in range(0, len(values), batch_size):
        batch = values[i:i + batch_size]
        worksheet.update(batch, f"A{i+1}")  # Новый порядок аргументов

#### 0.2. Автоматический выбор листа - START_TEST_10 !!!
Для парсинга разных сайтов/категорий:

In [11]:
#  - START_TEST_10 !!!
def infer_sheet_name(item: Dict) -> str:
    """Определяет имя листа на основе данных (например, по полю 'category')."""
    return item.get("category", "General").replace("/", "_")[:30]  # Ограничение Google

In [ ]:
# Пример использования:
# Создаст листы "VC_RU", "RBC" и т.д.
# export_to_google_sheets(data, sheet_name="auto")

export_to_google_sheets(
    data_source="../data/articles.json",
    spreadsheet_id="ваш_id_таблицы",
    sheet_name="auto"
)

#### 0.3. Пакетная запись (batch update) - START_TEST_11 !!!
Избегаем ограничений Google Sheets API:

In [12]:
#  - START_TEST_11 !!!
def update_worksheet(worksheet, data: List[Dict], batch_size: int) -> None:
    """Обновляет лист пачками (совместимость с gspread >=4.0)."""
    headers = list(data[0].keys())
    values = [headers] + [list(item.values()) for item in data]
    
    for i in range(0, len(values), batch_size):
        batch = values[i:i + batch_size]
        worksheet.update(batch, f"A{i+1}")  # Новый порядок аргументов

#### 0.4. Расширенная дедупликация - START_TEST_12 !!!

In [34]:
#  - START_TEST_12 !!!
def merge_and_deduplicate(old: List[Dict], new: List[Dict], key: str) -> List[Dict]:
    """
    Объединяет данные, сохраняя ПЕРВУЮ дату появления записи.
    
    Args:
        old: Старые данные из Google Sheets.
        new: Новые данные из парсинга.
        key: Поле для дедупликации (например, 'title').

        Сначала обрабатываются старые данные, чтобы их даты имели приоритет.
        При обновлении записей сохраняется оригинальная дата из old.
        
    Returns:
        Список словарей без дубликатов с оригинальными датами.
    """
    # Собираем все записи в словарь {ключ: данные}
    combined = {}
    
    # Сначала добавляем старые данные (чтобы их даты были приоритетными)
    for item in old:
        if item[key] not in combined:
            combined[item[key]] = item
    
    # Затем обновляем новыми данными, но сохраняем старые даты
    for item in new:
        if item[key] in combined:
            # Сохраняем старую дату из combined
            old_date = combined[item[key]].get("parse_datetime")
            # Обновляем запись, но оставляем старую дату
            combined[item[key]].update(item)
            combined[item[key]]["parse_datetime"] = old_date
        else:
            combined[item[key]] = item
    
    return list(combined.values())

In [36]:
# Проверка работы функции merge_and_deduplicate()

old = [{"title": "A", "parse_datetime": "2025-01-01"}]
new = [{"title": "A", "parse_datetime": "2025-05-10"}]
result = merge_and_deduplicate(old, new, "title")
print(result[0]["parse_datetime"])  # Выведет 2025-01-01

2025-01-01


#### 0.5. Проверка существования листа (или создание нового при отсутствии) get_or_create_worksheet() - START_TEST_13 !!!

In [14]:
# Эта функция проверяет существование листа и создаёт новый при необходимости: - START_TEST_13 !!!
def get_or_create_worksheet(spreadsheet, sheet_name: str):
    """
    Находит или создаёт лист в Google Sheets.
    
    Args:
        spreadsheet: Объект таблицы (gspread.Spreadsheet)
        sheet_name: Название листа
        
    Returns:
        Объект листа (gspread.Worksheet)
    """
    try:
        # Пытаемся получить лист
        worksheet = spreadsheet.worksheet(sheet_name)
        return worksheet
    except gspread.exceptions.WorksheetNotFound:
        # Создаём новый лист, если не найден
        worksheet = spreadsheet.add_worksheet(
            title=sheet_name,
            rows=100,
            cols=20
        )
        return worksheet

#### 0.6. Форматирование дат format_dates() - START_TEST_14 !!!

In [15]:
# Форматирует даты во всех записях: - START_TEST_14 !!!
from datetime import datetime

def format_dates(data: List[Dict], time_format: str) -> None:
    """
    Приводит все даты в данных к единому формату.
    
    Args:
        data: Список словарей с данными
        time_format: Целевой формат (например, "%d.%m.%Y %H:%M")
    """
    for item in data:
        if 'date' in item and item['parse_datetime']:
            try:
                # Пробуем распарсить дату из разных форматов
                if isinstance(item['parse_datetime'], str):
                    parsed_date = datetime.strptime(item['date'], "%Y-%m-%d %H:%M:%S")
                else:
                    parsed_date = item['parse_datetime']
                item['parse_datetime'] = parsed_date.strftime(time_format)
            except (ValueError, TypeError):
                item['parse_datetime'] = "N/A"  # Если дата некорректна

### 1. Улучшенная версия функции - START_TEST_11 !!! - START_8 !!!

#### Ключевые моменты работы функции
1. Авторизация:
Credentials.from_service_account_file() — загружает учётные данные из JSON-файла.
gspread.authorize() — создаёт клиент для работы с API.

2. Дедупликация:
merge_and_deduplicate() объединяет старые и новые данные, перезаписывая дубликаты.

3. Пакетная запись:
update_worksheet() отправляет данные порциями (избегает ограничений API Google).

4. Обработка ошибок:
Весь код обёрнут в try-except для логирования ошибок.

In [16]:
# прежняя версия
import gspread
from google.oauth2.service_account import Credentials
from typing import List, Dict, Optional, Union
import logging
from datetime import datetime

def export_to_google_sheets(
    data_source: Union[str, List[Dict]],
    spreadsheet_id: str,
    sheet_name: str = "News",
    credentials_path: str = "../credentials.json",
    dedup_key: str = "title",
    batch_size: int = 1000,
    time_format: str = "%d.%m.%Y %H:%M"
) -> Optional[int]:
    """
    Экспортирует данные в Google Sheets с дедупликацией.
    
    Args:
        data_source: Путь к JSON-файлу или список словарей
        spreadsheet_id: ID таблицы (из URL)
        sheet_name: Название листа ('auto' для автовыбора)
        credentials_path: Путь к файлу учётных данных Google
        dedup_key: Поле для дедупликации (по умолчанию 'title')
        batch_size: Максимальное количество строк за один запрос
        time_format: Формат даты (например, "%d.%m.%Y %H:%M")
        
    Returns:
        Количество уникальных записей или None при ошибке
    """
    try:
        # --- Инициализация клиента Google Sheets ---
        creds = Credentials.from_service_account_file(credentials_path)
        gc = gspread.authorize(creds)  # Авторизация
        spreadsheet = gc.open_by_key(spreadsheet_id)  # Открываем таблицу

        # --- Загрузка данных ---
        if isinstance(data_source, str):
            # Если передан путь к JSON
            with open(data_source, 'r', encoding='utf-8') as f:
                new_data = json.load(f)
        else:
            # Если передан готовый список словарей
            new_data = data_source

        if not new_data:
            logging.warning("Нет данных для экспорта.")
            return 0

        # --- Автовыбор имени листа ---
        if sheet_name == "auto":
            sheet_name = infer_sheet_name(new_data[0])  # Берём категорию из данных

        # --- Работа с листом ---
        worksheet = get_or_create_worksheet(spreadsheet, sheet_name)
        
        # --- Дедупликация ---
        old_data = worksheet.get_all_records()  # Читаем старые данные
        unique_data = merge_and_deduplicate(old_data, new_data, dedup_key)
        
        # --- Форматирование дат ---
        format_dates(unique_data, time_format)
        
        # --- Пакетная запись ---
        update_worksheet(worksheet, unique_data, batch_size)
        
        logging.info(f"Экспортировано записей: {len(unique_data)}")
        return len(unique_data)
        
    except Exception as e:
        logging.error(f"Ошибка экспорта: {e}", exc_info=True)
        return None

In [13]:
# объединённая версия - START_TEST_11  - START_8 !!!
import gspread
import json
import logging
from google.oauth2.service_account import Credentials
from typing import List, Dict, Optional, Union
from datetime import datetime

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("../logs/google_sheets_errors.log"),
        logging.StreamHandler()
    ]
)

def export_to_google_sheets(
    data_source: Union[str, List[Dict]],
    spreadsheet_id: str,
    sheet_name: str = "News",
    credentials_path: str = "../credentials.json",
    dedup_key: str = "title",
    batch_size: int = 1000,
    time_format: str = "%d.%m.%Y %H:%M:%S"
) -> Optional[int]:
    """
    Экспортирует данные в Google Sheets с дедупликацией.
    
    Args:
        data_source: Путь к JSON-файлу или список словарей
        spreadsheet_id: ID таблицы (из URL)
        sheet_name: Название листа ('auto' для автовыбора)
        credentials_path: Путь к файлу учётных данных Google
        dedup_key: Поле для дедупликации (по умолстанию 'title')
        batch_size: Максимальное количество строк за один запрос
        time_format: Формат даты (например, "%d.%m.%Y %H:%M")
        
    Returns:
        Количество уникальных записей или None при ошибке
    """
    try:
        # --- Инициализация клиента Google Sheets с правильными scopes ---
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        creds = Credentials.from_service_account_file(credentials_path, scopes=scopes)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open_by_key(spreadsheet_id)

        # --- Загрузка данных ---
        if isinstance(data_source, str):
            with open(data_source, 'r', encoding='utf-8') as f:
                new_data = json.load(f)
        else:
            new_data = data_source

        if not new_data:
            logging.warning("Нет данных для экспорта.")
            return 0

        # --- Автовыбор имени листа ---
        if sheet_name == "auto":
            sheet_name = infer_sheet_name(new_data[0])

        # --- Работа с листом ---
        worksheet = get_or_create_worksheet(spreadsheet, sheet_name)
        
        # --- Дедупликация ---
        old_data = worksheet.get_all_records()
        unique_data = merge_and_deduplicate(old_data, new_data, dedup_key)
        count_old_data = len(old_data)
        count_unique_data = len(unique_data)
        count_new_data = len(unique_data)-len(old_data)
        
        # --- Форматирование дат ---
        # format_dates(unique_data, time_format)
        
        # --- Пакетная запись ---
        update_worksheet(worksheet, unique_data, batch_size)
        
        logging.info(f"Старых записей: {count_old_data}. Новых записей: {count_new_data}. Экспортировано записей: {count_unique_data}.")
        logging.info(f"Парсинг отработал успешно !!\n")
        print("✅ 3. Google Sheets дополнена новым контентом!")
        print("\tСкрипт отработал:", datetime.now().strftime('%d.%m.%Y %H:%M:%S'))
        return count_old_data, count_unique_data, count_new_data
        
    except Exception as e:
        logging.error(f"Ошибка экспорта: {e}", exc_info=True)
        return None

In [14]:
# Вариант 1: Из файла
export_to_google_sheets("../data/output/articles.json", "123abc")
# export_to_google_sheets("../data/output/articles.json")

ERROR:root:Ошибка экспорта: <Response [404]>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\gspread\client.py", line 155, in open_by_key
    spreadsheet = Spreadsheet(self.http_client, {"id": key})
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\gspread\spreadsheet.py", line 29, in __init__
    metadata = self.fetch_sheet_metadata()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\gspread\spreadsheet.py", line 230, in fetch_sheet_metadata
    return self.client.fetch_sheet_metadata(self.id, params=params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\gspread\http_client.py", line 305, in fetch_sheet_metadata
    r = self.request("get", url, params=params)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# Вариант 2: Напрямую из памяти
articles = parse_articles(html)
export_to_google_sheets(articles, "123abc")

In [16]:
# Вариант 3: Пример вызова как в файле PARSUNG_WITH_SIMYCH
export_to_google_sheets(
    data_source="../data/output/articles.json",
    spreadsheet_id="1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
    sheet_name="VC.ru News"
)

# export_to_google_sheets(
#     data_source="../data/output/articles.json",
#     spreadsheet_id="1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
#     sheet_name="auto"
# )

✅ 3. Google Sheets дополнена новым контентом!
	Скрипт отработал: 12.05.2025 07:59:50


(246, 0)

### 2. Ключевые улучшения

#### 2.1. Поддержка разных источников данных

Теперь функция принимает:
- Путь к JSON (data/output/articles.json)
- Готовый список словарей (результат parse_articles())

In [ ]:
# 
# Вариант 1: Из файла
export_to_google_sheets("../data/output/articles.json", "123abc")

# Вариант 2: Напрямую из памяти
articles = parse_articles(html)
export_to_google_sheets(articles, "123abc")

### 3. Интеграция с многоплатформенным парсингом

#### 3.1. Стратегия для разных сайтов
Подход	       Пример листа	    Метод вызова
Один лист	   All_News	        sheet_name="All_News"
По сайтам	   VC_RU, RBC	    sheet_name="auto"
По категориям  Marketing, Tech	sheet_name=item["category"]

#### 3.2. Пример для VC.ru + RBC

In [ ]:
# Парсим VC.ru
vc_data = parse_articles(vc_html, host="https://vc.ru")
export_to_google_sheets(vc_data, "123abc", sheet_name="auto")

# Парсим RBC
rbc_data = parse_articles(rbc_html, host="https://rbc.ru")
export_to_google_sheets(rbc_data, "123abc", sheet_name="auto")

### 4. Дополнительные улучшения

#### 4.1. Валидация данных

In [ ]:
# 
def validate_data(data: List[Dict]) -> bool:
    """Проверяет обязательные поля."""
    required_fields = {"title", "link", "date"}
    return all(required_fields <= set(item.keys()) for item in data)

#### 4.2. Логирование изменений

In [ ]:
# 
def log_changes(added: int, updated: int, sheet_name: str) -> None:
    """Пишет историю изменений в отдельный лист."""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "sheet": sheet_name,
        "action": f"Added: {added}, Updated: {updated}"
    }
    # Запись в лист 'Logs'
    ...

#### 4.3. Обновление только новых данных

In [ ]:
# Оптимизация для больших таблиц:
last_update = worksheet.acell("A1").value  # Дата последнего обновления
new_items = [item for item in data if item["date"] > last_update]

#### Проверка библиотеки GSPREAD

In [24]:
!pip show gspread

Name: gspread
Version: 6.1.4
Summary: Google Spreadsheets Python API
Home-page: 
Author: 
Author-email: Anton Burnashev <fuss.here@gmail.com>
License: 
Location: C:\Users\User\AppData\Roaming\Python\Python312\site-packages
Requires: google-auth, google-auth-oauthlib
Required-by: 


### 5. Пример итоговой структуры Google Sheets

Лист	Назначение
VC_RU	Новости с vc.ru
RBC 	Новости с rbc.ru
Logs	История обновлений
Config	Настройки (например, dedup_key)

### 6. Рекомендации

1. Разделение данных:
Используйте отдельные листы, если источники/категории не связаны между собой.
Для общего анализа удобнее один лист с полем source.

2. Безопасность:
Храните credentials.json вне репозитория (добавьте в .gitignore).
Ограничьте права сервисного аккаунта в Google Cloud.

3. Производительность:
Для больших объёмов данных (>10K строк) используйте Google Sheets API batchUpdate.

## 6. START_PARSING - START_9 !!!

In [ ]:
# Функция запуска парсинга (основная объединяющая функция)

### 3. Механизм возобновления работы - START_9 !!!

In [10]:
#  - START_9 !!!
import pickle
from pathlib import Path

def save_failure_state(resume_point: str, data: Optional[List[Dict]] = None) -> None:
    """Сохраняет состояние для возобновления."""
    state = {
        'resume_from': resume_point,
        'data': data,
        'timestamp': datetime.now().isoformat()
    }
    
    Path("../data/state").mkdir(exist_ok=True)
    with open("../data/state/last_failure.pkl", "wb") as f:
        pickle.dump(state, f)

def load_failure_state() -> Optional[Dict]:
    """Загружает последнее состояние при ошибке."""
    try:
        with open("../data/state/last_failure.pkl", "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

In [ ]:
# 


### 4. Дополнительные улучшения

#### 4.1. Проверка состояния при запуске
Добавьте в начало start_parsing():

In [ ]:
# 
# Автоматическое возобновление
if not resume_from:
    last_state = load_failure_state()
    if last_state:
        choice = input(f"Обнаружено незавершённое выполнение (ошибка на этапе '{last_state['resume_from']}'). Продолжить? (y/n): ")
        if choice.lower() == 'y':
            resume_from = last_state['resume_from']
            articles_data = last_state['data']

#### 4.2. Промежуточные статусы

In [ ]:
# Пример функции для отправки статусов:
def send_status_update(chat_id: str, step: str, details: str) -> None:
    """Отправляет промежуточный статус."""
    steps = {
        'dynamic_parse': "🔄 Динамическая загрузка страницы",
        'parse_articles': "📝 Парсинг новостей",
        'update_data': "💾 Сохранение данных",
        'export': "📤 Экспорт в Google Sheets"
    }
    
    send_telegram_message(
        chat_id=chat_id,
        text=f"{steps.get(step, '🚀')} {details}",
        action="status"
    )

### 2. Улучшенные Telegram-уведомления - START_10 !!!

In [10]:
# токен лежит в файле config.py - START_10 !!!
import config

def send_telegram_message(
    chat_id: str,
    text: str,
    action: str = "info",
    spreadsheet_url: Optional[str] = None,
    error_info: Optional[Dict] = None
) -> None:
    """
    Отправляет интерактивные сообщения в Telegram.
    
    Args:
        action: Тип действия ('start', 'finish', 'error', 'status')
    """
    bot = telebot.TeleBot(config.token)
    markup = None
    
    # Кнопки для разных сценариев
    if action == "finish" and spreadsheet_url:
        markup = InlineKeyboardMarkup()
        markup.add(InlineKeyboardButton("📊 Открыть Google Sheets", url=spreadsheet_url))
        
    elif action == "error":
        markup = InlineKeyboardMarkup()
        markup.add(
            InlineKeyboardButton("🔄 Повторить попытку", callback_data="retry"),
            InlineKeyboardButton("🔍 Подробности", callback_data=f"error_details:{error_info['resume_point']}")
        )
    
    # Отправка сообщения
    try:
        bot.send_message(
            chat_id,
            text,
            reply_markup=markup,
            parse_mode="Markdown"
        )
    except Exception as e:
        logging.error(f"Ошибка отправки в Telegram: {e}")
    finally:
        bot.stop_polling()

bot = telebot.TeleBot(config.token)
# Обработчик кнопок
@bot.callback_query_handler(func=lambda call: True)
def handle_buttons(call):
    if call.data == "retry":
        bot.send_message(call.message.chat.id, "Повторяю парсинг...")
        start_parsing(resume_from=call.message.text.split("'")[-2])
    elif call.data.startswith("error_details"):
        error_point = call.data.split(":")[1]
        bot.send_message(
            call.message.chat.id,
            f"Последняя точка сохранения: `{error_point}`\n"
            f"Для возобновления выполните:\n```python\nstart_parsing(resume_from='{error_point}')\n```",
            parse_mode="Markdown"
        )

In [ ]:
# Исправленный код функций

In [11]:
# Для send_telegram_message(): - START_10 !!!
def send_telegram_message(
    chat_id: str,
    text: str,
    action: str = "info",
    spreadsheet_url: Optional[str] = None,
    error_info: Optional[Dict] = None
) -> None:
    """
    Отправляет интерактивные сообщения в Telegram.
    """
    try:
        # Используем глобальный экземпляр бота
        global bot
        
        markup = None
        if action == "finish" and spreadsheet_url:
            markup = InlineKeyboardMarkup()
            markup.add(InlineKeyboardButton("📊 Открыть Google Sheets", url=spreadsheet_url))
        elif action == "error":
            markup = InlineKeyboardMarkup()
            markup.add(
                InlineKeyboardButton("🔄 Повторить", callback_data=f"retry:{error_info['resume_point']}"),
                InlineKeyboardButton("❌ Отмена", callback_data="cancel")
            )
        
        # Для статусных сообщений не используем разметку
        bot.send_message(
            chat_id,
            text,
            reply_markup=markup,
            parse_mode="Markdown"
        )
        
    except Exception as e:
        logging.error(f"Telegram send error: {e}")



In [ ]:
# Обработчик

In [ ]:
# 
from telebot import TeleBot, types
import config

bot = TeleBot(config.token)

@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    try:
        if call.data.startswith("retry"):
            _, resume_point = call.data.split(":")
            bot.send_message(call.message.chat.id, f"🔄 Перезапускаю с этапа: {resume_point}")
            start_parsing(resume_from=resume_point)
        elif call.data == "cancel":
            bot.send_message(call.message.chat.id, "❌ Парсинг отменён")
    except Exception as e:
        logging.error(f"Callback error: {e}")

### 1. Улучшенная start_parsing() с обработкой ошибок - START_11 !!!

In [25]:
#  - START_11 !!!
from typing import Optional, Dict
import traceback
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

def start_parsing(resume_from: Optional[str] = None) -> Dict[str, int]:
    """
    Основная функция парсинга с возможностью возобновления.
    
    Args:
        resume_from: Точка возобновления ('dynamic_parse', 'parse_articles', etc.)
        
    Returns:
        Словарь с результатами: {
            'total_articles': int,
            'unique_articles': int,
            'spreadsheet_url': str
        }
    """
    # Конфигурация
    config = {
        'chat_id': "1938719365",
        'host': "https://vc.ru",
        'url': "https://vc.ru/new",
        'spreadsheet_id': "1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
        'max_scrolls': 5,
        'max_retries': 3,
        'retry_delay': 5
    }
    
    # Результаты
    result = {
        'total_articles': 0,
        'unique_articles': 0,
        'spreadsheet_url': f"https://docs.google.com/spreadsheets/d/{config['spreadsheet_id']}"
    }
    
    try:
        # Автоматическое возобновление
        if not resume_from:
            last_state = load_failure_state()
            if last_state:
                choice = input(f"Обнаружено незавершённое выполнение (ошибка на этапе '{last_state['resume_from']}'). Продолжить? (y/n): ")
                if choice.lower() == 'y':
                    resume_from = last_state['resume_from']
                    articles_data = last_state['data']
        
        # Этап 1: Стартовое сообщение
        if not resume_from:
            send_telegram_message(
                chat_id=config['chat_id'],
                text=f"🚀 Запуск парсера VC.ru\nВремя: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
                action="start"
            )

        # Этап 2: Динамический парсинг
        if not resume_from or resume_from == 'dynamic_parse':
            html = dynamic_parse(
                url=config['url'],
                max_scrolls=config['max_scrolls'],
                max_retries=config['max_retries'],
                retry_delay=config['retry_delay']
            )
            if html:
                with open("../data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
                    f.write(html)
                print("✅ 1. HTML сохранён с динамическим контентом!") # только для отладки
                print("\tСкрипт отработал:", datetime.now().strftime('%d.%m.%Y %H:%M:%S')) # только для отладки
                send_telegram_message(
                    chat_id=config['chat_id'],
                    text=f"✅ DYNAMIC_PARSE\nВремя: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
                    action="status"
                )
            else:
                logging.error("Не удалось загрузить страницу после нескольких попыток")
                
            resume_from = None  # Сброс после успешного 

        # Этап 3: Парсинг статей
        if not resume_from or resume_from == 'parse_articles':
            parser = VcRuParser(host=config['host'])
            articles_data = parser.parse_articles(html)
            result['total_articles'] = len(articles_data)
            send_telegram_message(
                chat_id=config['chat_id'],
                text=f"✅ PARSE_ARTICLES\nВремя: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
                action="status"
            )
            resume_from = None

        # Этап 4: Обновление данных
        if not resume_from or resume_from == 'update_data':
            update_data(articles_data)
            send_telegram_message(
                chat_id=config['chat_id'],
                text=f"✅ UPDATE_DATA\nВремя: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
                action="status"
            )
            resume_from = None

        # Этап 5: Экспорт в Google Sheets
        if not resume_from or resume_from == 'export_to_google_sheets':
            result['unique_articles'] = export_to_google_sheets(
                data_source=articles_data,
                spreadsheet_id=config['spreadsheet_id'],
                sheet_name="VC.ru News"
            )
            send_telegram_message(
                chat_id=config['chat_id'],
                text=f"✅ GOOGLE_SHEETS\nВремя: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
                action="status"
            )
            resume_from = None
        
        # Успешное завершение
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"✅ Парсинг завершен!\nВремя: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}\nУникальных записей: {result['unique_articles'][0]}.\nНовых записей: {result['unique_articles'][1]}.",
            action="finish",
            spreadsheet_url=result['spreadsheet_url']
        )
        
        return result

    except Exception as e:
        error_info = {
            'error': str(e),
            'traceback': traceback.format_exc(),
            'resume_point': resume_from or 'dynamic_parse'
        }
        
        # Отправка ошибки в Telegram
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"🔥 Ошибка на этапе '{resume_from or 'dynamic_parse'}': {e}",
            action="error",
            error_info=error_info
        )
        
        # Сохранение состояния для возобновления
        save_failure_state(resume_from, articles_data if 'articles_data' in locals() else None)
        raise  # Можно убрать для автоматического продолжения при следующем запуске

#### 5. Пример использования - START_12 !!!
##### 5.1. Ручной запуск

In [26]:
#  - START_12 !!!
if __name__ == "__main__":
    start_parsing()  # Обычный запуск
    # Или с возобновлением:
    # start_parsing(resume_from='parse_articles')

Обнаружено незавершённое выполнение (ошибка на этапе 'None'). Продолжить? (y/n):  n


2025-05-11 20:36:14,980 - __main__ - INFO - Начало парсинга: https://vc.ru/new
2025-05-11 20:38:23,486 - __main__ - INFO - Успешно завершено. Загружено страниц: 5


✅ 1. HTML сохранён с динамическим контентом!
	Скрипт отработал: 11.05.2025 20:38:26


2025-05-11 20:38:27,698 - root - ERROR - Ошибка отправки в Telegram: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: can't parse entities: Can't find end of the entity starting at byte offset 11
2025-05-11 20:38:46,439 - root - ERROR - Ошибка отправки в Telegram: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: can't parse entities: Can't find end of the entity starting at byte offset 9


✅ 2. Новые данные сохранёны в переменной ARTICLES для обработки!
	Скрипт отработал: 11.05.2025 20:38:46


2025-05-11 20:39:03,231 - __main__ - ERROR - Ошибка в update_data: time data "12.04.2025. 17:41:07" doesn't match format "%d.%m.%Y %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_6508\2329151264.py", line 51, in update_data
    old_df = load_existing_data(csv_path, date_format)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_6508\2329151264.py", line 82, in load_existing_data
    df["parse_datetime"] = pd.to_datetime(
                           ^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\tools

✅ 3. Google Shets дополнена новым контентом!
	Скрипт отработал: 2025-05-11 20:39:16


#### 5.2. Через планировщик задач

In [ ]:
# Для Windows (run.bat):
# bat
@echo off
call venv\Scripts\activate
python -c "from main import start_parsing; start_parsing()"

### 6. Итоговая архитектура уведомлений
Тип сообщения	Кнопки	               Пример текста
Старт	        -	                   "🚀 Начало парсинга VC.ru в 12:00"
Ошибка	        Повторить/Подробности  "🔥 Ошибка парсинга: timeout"
Статус	        -	                   "📝 Парсинг: 42 новости обработаны"
Завершение	    Открыть Google Sheets  "✅ Успешно! 100 записей"

### Рекомендации
1. Логирование:
Сохраняйте полный лог в файл logs/parsing_<date>.log.

2. Конфигурация:
Вынесите chat_id, spreadsheet_id в .env или config.py.

3. Тестирование:
Добавьте mock-тесты для Telegram API.

### Эти изменения дают:

✅ Возобновление после сбоев
✅ Подробные уведомления в Telegram
✅ Интерактивные элементы управления
✅ Гибкую конфигурацию

## 7. Переработанная функция START_PARSING() - START_10

### 3. Механизм возобновления работы - START_9 !!!

In [12]:
#  - START_9 !!!
import pickle
from pathlib import Path

def save_failure_state(resume_point: str, data: Optional[List[Dict]] = None) -> None:
    """Сохраняет состояние для возобновления."""
    state = {
        'resume_from': resume_point,
        'data': data,
        'timestamp': datetime.now().isoformat()
    }
    
    Path("../data/state").mkdir(exist_ok=True)
    with open("../data/state/last_failure.pkl", "wb") as f:
        pickle.dump(state, f)

def load_failure_state() -> Optional[Dict]:
    """Загружает последнее состояние при ошибке."""
    try:
        with open("../data/state/last_failure.pkl", "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

#### 4.2. Промежуточные статусы

In [19]:
# Пример функции для отправки статусов:
def send_status_update(chat_id: str, step: str, details: str) -> None:
    """Отправляет промежуточный статус."""
    steps = {
        'dynamic_parse': "🔄 Динамическая загрузка страницы",
        'parse_articles': "📝 Парсинг новостей",
        'update_data': "💾 Сохранение данных",
        'export': "📤 Экспорт в Google Sheets"
    }
    
    send_telegram_message(
        chat_id=chat_id,
        text=f"{steps.get(step, '🚀')} {details}",
        action="status"
    )

### 2. Улучшенные Telegram-уведомления - START_10 !!!

In [13]:
# Для send_telegram_message(): - START_10 !!!
def send_telegram_message(
    chat_id: str,
    text: str,
    action: str = "info",
    spreadsheet_url: Optional[str] = None,
    error_info: Optional[Dict] = None
) -> None:
    """
    Отправляет интерактивные сообщения в Telegram.
    """
    try:
        # Используем глобальный экземпляр бота
        global bot
        
        markup = None
        if action == "finish" and spreadsheet_url:
            markup = InlineKeyboardMarkup()
            markup.add(InlineKeyboardButton("📊 Открыть Google Sheets", url=spreadsheet_url))
        elif action == "error":
            markup = InlineKeyboardMarkup()
            markup.add(
                InlineKeyboardButton("🔄 Повторить", callback_data=f"retry:{error_info['resume_point']}"),
                InlineKeyboardButton("❌ Отмена", callback_data="cancel")
            )
        
        # Для статусных сообщений не используем разметку
        bot.send_message(
            chat_id,
            text,
            reply_markup=markup,
            parse_mode="Markdown"
        )
        
    except Exception as e:
        logging.error(f"Telegram send error: {e}")



In [21]:
# Обработчик

In [22]:
# 
from telebot import TeleBot, types
import config

bot = TeleBot(config.token)

@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    try:
        if call.data.startswith("retry"):
            _, resume_point = call.data.split(":")
            bot.send_message(call.message.chat.id, f"🔄 Перезапускаю с этапа: {resume_point}")
            start_parsing(resume_from=resume_point)
        elif call.data == "cancel":
            bot.send_message(call.message.chat.id, "❌ Парсинг отменён")
    except Exception as e:
        logging.error(f"Callback error: {e}")

### start_parsing() с интеграцией всех исправлений и подробными комментариями: - START_11 !!!

In [14]:
#  - START_11 !!!
import threading
from telebot import TeleBot, types
import config
import logging
from datetime import datetime
from typing import Optional, Dict

# Инициализация бота (глобальный экземпляр)
bot = TeleBot(config.token)

# Запуск бота в отдельном потоке
def run_bot():
    """Запускает бота в фоновом режиме"""
    try:
        bot.infinity_polling()
    except Exception as e:
        logging.error(f"Bot polling error: {e}")

bot_thread = threading.Thread(target=run_bot, daemon=True)
bot_thread.start()

def start_parsing(resume_from: Optional[str] = None) -> Dict[str, int]:
    """
    Основная функция парсинга с Telegram-уведомлениями.
    
    Args:
        resume_from: Точка возобновления ('dynamic_parse', 'parse_articles', etc.)
        
    Returns:
        Словарь с результатами: {
            'total_articles': int,
            'unique_articles': int,
            'spreadsheet_url': str
        }
    """
    # Конфигурация (вынесена в отдельный файл config.py в реальном проекте)
    config = {
        'chat_id': "1938719365",
        'host': "https://vc.ru",
        'url': "https://vc.ru/new",
        'spreadsheet_id': "1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
        'max_scrolls': 5,
        'max_retries': 3,
        'retry_delay': 5
    }
    
    # Результаты
    result = {
        'total_articles': 0,
        'unique_articles': 0,
        'spreadsheet_url': f"https://docs.google.com/spreadsheets/d/{config['spreadsheet_id']}"
    }
    
    try:
        # --- Этап 1: Инициализация ---
        if not resume_from:
            send_status_update(config['chat_id'], "start", f"\n{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}\nЗапуск парсера VC.ru")
            
            # Проверка незавершенных задач
            last_state = load_failure_state()
            if last_state:
                msg = f"Обнаружен незавершенный парсинг (этап: {last_state['resume_from']}). Продолжить?"
                if ask_confirmation(config['chat_id'], msg):
                    resume_from = last_state['resume_from']
                    articles_data = last_state['data']

        # --- Этап 2: Динамический парсинг ---
        if not resume_from or resume_from == 'dynamic_parse':
            send_status_update(config['chat_id'], "dynamic_parse", "Загрузка страницы...")
            
            html = dynamic_parse(
                url=config['url'],
                max_scrolls=config['max_scrolls'],
                max_retries=config['max_retries'],
                retry_delay=config['retry_delay']
            )
            
            if not html:
                raise Exception("Не удалось загрузить HTML")
                
            with open("../data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
                f.write(html)
                
            send_status_update(config['chat_id'], "dynamic_parse", f"✅ HTML успешно сохранен\n{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")
            resume_from = None

        # --- Этап 3: Парсинг статей ---
        if not resume_from or resume_from == 'parse_articles':
            send_status_update(config['chat_id'], "parse_articles", "Обработка новостей...")
            
            parser = VcRuParser(host=config['host'])
            articles_data = parser.parse_articles(html)
            result['total_articles'] = len(articles_data)
            
            send_status_update(config['chat_id'], "parse_articles", f"✅ Найдено статей: {result['total_articles']}")
            resume_from = None

        # --- Этап 4: Обновление данных ---
        if not resume_from or resume_from == 'update_data':
            send_status_update(config['chat_id'], "update_data", "Сохранение данных...")
            
            update_data(articles_data)
            send_status_update(config['chat_id'], "update_data", "✅ Данные обновлены")
            resume_from = None

        # --- Этап 5: Экспорт в Google Sheets ---
        if not resume_from or resume_from == 'export_to_google_sheets':
            send_status_update(config['chat_id'], "export", "Экспорт в Google Sheets...")
            
            result['unique_articles'] = export_to_google_sheets(
                data_source=articles_data,
                spreadsheet_id=config['spreadsheet_id'],
                sheet_name="VC.ru News"
            )
            
            send_status_update(config['chat_id'], "export", f"✅ Экспортировано {result['unique_articles'][0]} записей")
            resume_from = None

        # --- Успешное завершение ---
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"✅ Парсинг завершен!\n"
                 f"• Новых записей: {result['unique_articles'][1]}\n"
                 f"• Уникальных записей: {result['unique_articles'][0]}\n"
                 f"{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
            action="finish",
            spreadsheet_url=result['spreadsheet_url']
        )
        
        return result

    except Exception as e:
        # --- Обработка ошибок ---
        error_info = {
            'error': str(e),
            'resume_point': resume_from or 'dynamic_parse',
            'timestamp': datetime.now().isoformat()
        }
        
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"⚠️ Ошибка: {str(e)}",
            action="error",
            error_info=error_info
        )
        
        save_failure_state(resume_from, articles_data if 'articles_data' in locals() else None)
        raise

# Вспомогательные функции для Telegram
def send_status_update(chat_id: str, step: str, message: str):
    """Отправляет статусное сообщение с форматированием"""
    step_names = {
        'start': "🚀 Начало работы",
        'dynamic_parse': "🔍 Динамический парсинг",
        'parse_articles': "📝 Анализ статей",
        'update_data': "💾 Сохранение данных",
        'export': "📤 Экспорт в Google Sheets"
    }
    
    text = f"{step_names.get(step, 'ℹ️ Этап')}: {message}"
    send_telegram_message(chat_id, text, action="status")

def ask_confirmation(chat_id: str, question: str) -> bool:
    """Запрашивает подтверждение через Telegram"""
    markup = types.InlineKeyboardMarkup()
    markup.add(
        types.InlineKeyboardButton("✅ Да", callback_data="confirm_yes"),
        types.InlineKeyboardButton("❌ Нет", callback_data="confirm_no")
    )
    
    msg = bot.send_message(chat_id, question, reply_markup=markup)
    
    # Ожидание ответа (упрощенная реализация)
    @bot.callback_query_handler(func=lambda call: call.message.message_id == msg.message_id)
    def handle_confirmation(call):
        if call.data == "confirm_yes":
            return True
        return False

# Обработчик callback'ов (должен быть в глобальной области видимости)
@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    try:
        if call.data.startswith("retry"):
            _, resume_point = call.data.split(":")
            bot.send_message(call.message.chat.id, f"🔄 Перезапускаю с этапа: {resume_point}")
            start_parsing(resume_from=resume_point)
        elif call.data == "cancel":
            bot.send_message(call.message.chat.id, "❌ Парсинг отменён")
    except Exception as e:
        logging.error(f"Callback error: {e}")

### 5. Пример использования - START_12 !!!
#### 5.1. Ручной запуск

In [15]:
#  - START_12 !!!
if __name__ == "__main__":
    start_parsing()  # Обычный запуск
    # Или с возобновлением:
    # start_parsing(resume_from='parse_articles')

2025-05-12 20:09:16,953 - WDM - INFO - ====== WebDriver manager ======
2025-05-12 20:09:59,613 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-05-12 20:10:00,183 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-05-12 20:10:00,596 - WDM - INFO - Driver [C:\Users\User\.wdm\drivers\chromedriver\win64\135.0.7049.114\chromedriver-win32/chromedriver.exe] found in cache
2025-05-12 20:11:23,784 - root - INFO - Прокрутка #1
2025-05-12 20:11:28,396 - root - INFO - Прокрутка #2
2025-05-12 20:11:35,363 - root - INFO - Прокрутка #3
2025-05-12 20:11:42,657 - root - INFO - Прокрутка #4
2025-05-12 20:11:50,533 - root - INFO - Прокрутка #5
2025-05-12 20:11:50,938 - root - INFO - Динамический контент успешно загружен. Количество прокруток: 5.


✅ 2. Новые данные сохранёны в переменной ARTICLES для обработки!
	Скрипт отработал: 12.05.2025 20:12:10


2025-05-12 20:12:12,098 - __main__ - INFO - Данные обновлены. Уникальных записей: 287. Новых записей: 34


✅ 3. CSV/JSON файлы сохранёны и дополнены новым контентом!
	Скрипт отработал: 12.05.2025 20:12:12


2025-05-12 20:12:23,639 - root - INFO - Экспортировано записей: 287. Новых записей: 34


✅ 3. Google Sheets дополнена новым контентом!
	Скрипт отработал: 12.05.2025 20:12:23


2025-05-12 20:23:59,509 (__init__.py:1115 Thread-5 (run_bot)) ERROR - TeleBot: "Infinity polling exception: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)"
2025-05-12 20:23:59,509 - TeleBot - ERROR - Infinity polling exception: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)
2025-05-12 20:23:59,591 (__init__.py:1117 Thread-5 (run_bot)) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\ProgramData\anaconda3\Lib\h

## 8. Исправляем ошибку ТелеБота START_PARSING() - START_10

### 3. Механизм возобновления работы - START_9 !!!

In [12]:
#  - START_9 !!!
import pickle
from pathlib import Path

def save_failure_state(resume_point: str, data: Optional[List[Dict]] = None) -> None:
    """Сохраняет состояние для возобновления."""
    state = {
        'resume_from': resume_point,
        'data': data,
        'timestamp': datetime.now().isoformat()
    }
    
    Path("../data/state").mkdir(exist_ok=True)
    with open("../data/state/last_failure.pkl", "wb") as f:
        pickle.dump(state, f)

def load_failure_state() -> Optional[Dict]:
    """Загружает последнее состояние при ошибке."""
    try:
        with open("../data/state/last_failure.pkl", "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

#### 4.2. Промежуточные статусы

In [19]:
# Пример функции для отправки статусов:
def send_status_update(chat_id: str, step: str, details: str) -> None:
    """Отправляет промежуточный статус."""
    steps = {
        'dynamic_parse': "🔄 Динамическая загрузка страницы",
        'parse_articles': "📝 Парсинг новостей",
        'update_data': "💾 Сохранение данных",
        'export': "📤 Экспорт в Google Sheets"
    }
    
    send_telegram_message(
        chat_id=chat_id,
        text=f"{steps.get(step, '🚀')} {details}",
        action="status"
    )

### 2. Улучшенные Telegram-уведомления - START_10 !!!

In [13]:
# Для send_telegram_message(): - START_10 !!!
def send_telegram_message(
    chat_id: str,
    text: str,
    action: str = "info",
    spreadsheet_url: Optional[str] = None,
    error_info: Optional[Dict] = None
) -> None:
    """
    Отправляет интерактивные сообщения в Telegram.
    """
    try:
        # Используем глобальный экземпляр бота
        global bot
        
        markup = None
        if action == "finish" and spreadsheet_url:
            markup = InlineKeyboardMarkup()
            markup.add(InlineKeyboardButton("📊 Открыть Google Sheets", url=spreadsheet_url))
        elif action == "error":
            markup = InlineKeyboardMarkup()
            markup.add(
                InlineKeyboardButton("🔄 Повторить", callback_data=f"retry:{error_info['resume_point']}"),
                InlineKeyboardButton("❌ Отмена", callback_data="cancel")
            )
        
        # Для статусных сообщений не используем разметку
        bot.send_message(
            chat_id,
            text,
            reply_markup=markup,
            parse_mode="Markdown"
        )
        
    except Exception as e:
        logging.error(f"Telegram send error: {e}")



In [21]:
# Обработчик

In [22]:
# 
from telebot import TeleBot, types
import config

bot = TeleBot(config.token)

@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    try:
        if call.data.startswith("retry"):
            _, resume_point = call.data.split(":")
            bot.send_message(call.message.chat.id, f"🔄 Перезапускаю с этапа: {resume_point}")
            start_parsing(resume_from=resume_point)
        elif call.data == "cancel":
            bot.send_message(call.message.chat.id, "❌ Парсинг отменён")
    except Exception as e:
        logging.error(f"Callback error: {e}")

### start_parsing() с учетом ошибок от телебота: - START_11 !!!

In [14]:
#  - START_11 !!!
import threading
from telebot import TeleBot, types
import config
import logging
from datetime import datetime
from typing import Optional, Dict
from telebot import apihelper
import telebot
from threading import Lock
import time
from functools import wraps

# Настройка таймаутов
apihelper.SESSION_TIME_TO_LIVE = 5 * 60  # 5 минут
apihelper.READ_TIMEOUT = 30
apihelper.CONNECT_TIMEOUT = 10

# Инициализация бота (глобальный экземпляр)
bot = telebot.TeleBot(config.token)
bot_running = True
send_lock = Lock()

# Отдельный модуль для Telegram
def safe_send_message(chat_id, text):
    with send_lock:
        try:
            bot.send_message(chat_id, text, timeout=10)
        except Exception as e:
            logging.error(f"Telegram send failed: {e}")

# Запуск бота в отдельном потоке
def run_bot():
    """Запускает бота в фоновом режиме"""
    while True:
        try:
            bot.infinity_polling()
        except Exception as e:
            logging.error(f"Bot error: {e}")
            time.sleep(5)

def bot_thread_func():
    while bot_running:
        try:
            bot.infinity_polling()
        except Exception as e:
            logging.error(f"Bot polling error: {e}")
            time.sleep(5)

def telegram_safe(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Telegram error: {e}")
    return wrapper

@telegram_safe
def send_telegram_message(chat_id, text):
    bot.send_message(chat_id, text, timeout=10)


def start_parsing(resume_from: Optional[str] = None) -> Dict[str, int]:
    global bot_running
    """
    Основная функция парсинга с Telegram-уведомлениями.
    
    Args:
        resume_from: Точка возобновления ('dynamic_parse', 'parse_articles', etc.)
        
    Returns:
        Словарь с результатами: {
            'total_articles': int,
            'unique_articles': int,
            'spreadsheet_url': str
        }
    """
    # Конфигурация (вынесена в отдельный файл config.py в реальном проекте)
    # config = {
    #     'chat_id': "1938719365",
    #     'host': "https://vc.ru",
    #     'url': "https://vc.ru/new",
    #     'spreadsheet_id': "1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
    #     'max_scrolls': 5,
    #     'max_retries': 3,
    #     'retry_delay': 5
    # }
    
    # Результаты
    result = {
        'total_articles': 0,
        'unique_articles': 0,
        # 'spreadsheet_url': f"https://docs.google.com/spreadsheets/d/{config['spreadsheet_id']}"
        'spreadsheet_url': f"https://docs.google.com/spreadsheets/d/{config.spreadsheet_id}"
    }
    
    try:
        # --- Этап 1: Инициализация ---
        if not resume_from:
            send_status_update(config.chat_id], "start", f"\n{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}\nЗапуск парсера VC.ru")
            
            # Проверка незавершенных задач
            last_state = load_failure_state()
            if last_state:
                msg = f"Обнаружен незавершенный парсинг (этап: {last_state['resume_from']}). Продолжить?"
                if ask_confirmation(config.chat_id, msg):
                    resume_from = last_state['resume_from']
                    articles_data = last_state['data']

        # --- Этап 2: Динамический парсинг ---
        if not resume_from or resume_from == 'dynamic_parse':
            send_status_update(config.chat_id, "dynamic_parse", "Загрузка страницы...")
            
            html = dynamic_parse(
                url=config.url,
                max_scrolls=config.max_scrolls,
                max_retries=config.max_retries,
                retry_delay=config.retry_delay
            )
            
            if not html:
                raise Exception("Не удалось загрузить HTML")
                
            with open("../data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
                f.write(html)
                
            send_status_update(config.chat_id, "dynamic_parse", f"✅ HTML успешно сохранен\n{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")
            resume_from = None

        # --- Этап 3: Парсинг статей ---
        if not resume_from or resume_from == 'parse_articles':
            send_status_update(config.chat_id, "parse_articles", "Обработка новостей...")
            
            parser = VcRuParser(host=config.host)
            articles_data = parser.parse_articles(html)
            result['total_articles'] = len(articles_data)
            
            send_status_update(config.chat_id, "parse_articles", f"✅ Найдено статей: {result['total_articles']}")
            resume_from = None

        # --- Этап 4: Обновление данных ---
        if not resume_from or resume_from == 'update_data':
            send_status_update(config['chat_id'], "update_data", "Сохранение данных...")
            
            update_data(articles_data)
            send_status_update(config['chat_id'], "update_data", "✅ Данные обновлены")
            resume_from = None

        # --- Этап 5: Экспорт в Google Sheets ---
        if not resume_from or resume_from == 'export_to_google_sheets':
            send_status_update(config['chat_id'], "export", "Экспорт в Google Sheets...")
            
            result['unique_articles'] = export_to_google_sheets(
                data_source=articles_data,
                spreadsheet_id=config['spreadsheet_id'],
                sheet_name="VC.ru News"
            )
            
            send_status_update(config['chat_id'], "export", f"✅ Экспортировано {result['unique_articles'][0]} записей")
            resume_from = None

        # --- Успешное завершение ---
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"✅ Парсинг завершен!\n"
                 f"• Новых записей: {result['unique_articles'][1]}\n"
                 f"• Уникальных записей: {result['unique_articles'][0]}\n"
                 f"{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}",
            action="finish",
            spreadsheet_url=result['spreadsheet_url']
        )
        
        return result

    except Exception as e:
        # --- Обработка ошибок ---
        error_info = {
            'error': str(e),
            'resume_point': resume_from or 'dynamic_parse',
            'timestamp': datetime.now().isoformat()
        }
        
        send_telegram_message(
            chat_id=config['chat_id'],
            text=f"⚠️ Ошибка: {str(e)}",
            action="error",
            error_info=error_info
        )
        
        save_failure_state(resume_from, articles_data if 'articles_data' in locals() else None)
        raise

    finally:
        bot_running = False
        try:
            bot.stop_polling()
        except:
            pass

# Вспомогательные функции для Telegram
def send_status_update(chat_id: str, step: str, message: str):
    """Отправляет статусное сообщение с форматированием"""
    step_names = {
        'start': "🚀 Начало работы",
        'dynamic_parse': "🔍 Динамический парсинг",
        'parse_articles': "📝 Анализ статей",
        'update_data': "💾 Сохранение данных",
        'export': "📤 Экспорт в Google Sheets"
    }
    
    text = f"{step_names.get(step, 'ℹ️ Этап')}: {message}"
    send_telegram_message(chat_id, text, action="status")

def ask_confirmation(chat_id: str, question: str) -> bool:
    """Запрашивает подтверждение через Telegram"""
    markup = types.InlineKeyboardMarkup()
    markup.add(
        types.InlineKeyboardButton("✅ Да", callback_data="confirm_yes"),
        types.InlineKeyboardButton("❌ Нет", callback_data="confirm_no")
    )
    
    msg = bot.send_message(chat_id, question, reply_markup=markup)
    
    # Ожидание ответа (упрощенная реализация)
    @bot.callback_query_handler(func=lambda call: call.message.message_id == msg.message_id)
    def handle_confirmation(call):
        if call.data == "confirm_yes":
            return True
        return False

# Обработчик callback'ов (должен быть в глобальной области видимости)
@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    try:
        if call.data.startswith("retry"):
            _, resume_point = call.data.split(":")
            bot.send_message(call.message.chat.id, f"🔄 Перезапускаю с этапа: {resume_point}")
            start_parsing(resume_from=resume_point)
        elif call.data == "cancel":
            bot.send_message(call.message.chat.id, "❌ Парсинг отменён")
    except Exception as e:
        logging.error(f"Callback error: {e}")

if __name__ == "__main__":
    bot_thread = threading.Thread(target=bot_thread_func, daemon=True)
    bot_thread.start()
    
    start_parsing()
    
    # Даем боту время завершить отправку
    time.sleep(2)

## 9. Исправленная версия START_PARSING() с подробными комментариями - START_10 !!!

#### Ключевые улучшения:
1. Упрощенное управление ботом:
- Используем bot.polling() вместо infinity_polling с контролируемым таймаутом
- Глобальный флаг bot_running для управления потоком

2. Безопасная отправка сообщений:
- Декоратор @telegram_safe автоматически обрабатывает ошибки
- Задержка между сообщениями предотвращает блокировку

3. Стабильное завершение:
- Четкая последовательность остановки в finally
- Дополнительные паузы для завершения операций

4. Улучшенное логирование:
- Подробные сообщения об ошибках с трассировкой
- Разделение временных и критических ошибок

### Механизм возобновления работы - START_9 !!!

In [9]:
#  - START_9 !!!
import pickle
from pathlib import Path

def save_failure_state(resume_point: str, data: Optional[List[Dict]] = None) -> None:
    """Сохраняет состояние для возобновления."""
    state = {
        'resume_from': resume_point,
        'data': data,
        'timestamp': datetime.now().isoformat()
    }
    
    Path("../data/state").mkdir(exist_ok=True)
    with open("../data/state/last_failure.pkl", "wb") as f:
        pickle.dump(state, f)

def load_failure_state() -> Optional[Dict]:
    """Загружает последнее состояние при ошибке."""
    try:
        with open("../data/state/last_failure.pkl", "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

In [10]:
#  - START_10 !!!
import threading
from telebot import TeleBot, types, apihelper
import config
import logging
from datetime import datetime
from typing import Optional, Dict
from telebot import apihelper
import telebot
from threading import Lock
import time
from functools import wraps

# --- Конфигурация Telegram Bot ---
# Устанавливаем таймауты для устойчивого соединения
apihelper.SESSION_TIME_TO_LIVE = 5 * 60  # 5 минут жизни сессии
apihelper.READ_TIMEOUT = 30  # Таймаут чтения (сек)
apihelper.CONNECT_TIMEOUT = 10  # Таймаут соединения (сек)

# Инициализация бота (глобальная переменная)
bot = TeleBot(config.token)
bot_running = True  # Флаг для управления потоком бота

# --- Безопасная отправка сообщений ---
def telegram_safe(func):
    """Декоратор для безопасной отправки сообщений с обработкой ошибок"""
    @wraps(func)
    def wrapper(chat_id, text, *args, **kwargs):
        try:
            # Ограничение частоты отправки (1 сообщение/сек)
            time.sleep(1)
            return func(chat_id, text, *args, **kwargs)
        except Exception as e:
            logging.error(f"Telegram send error: {e}")
    return wrapper

@telegram_safe
def send_telegram_message(chat_id: str, text: str, **kwargs):
    """Безопасная отправка сообщения с обработкой ошибок"""
    bot.send_message(chat_id, text, **kwargs)

# --- Поток для работы бота ---
def bot_thread_func():
    """Функция для работы бота в отдельном потоке"""
    while bot_running:
        try:
            # Используем polling с обработкой сетевых ошибок
            bot.polling(none_stop=True, timeout=30)
        except Exception as e:
            logging.error(f"Bot polling error: {e}")
            time.sleep(5)  # Пауза перед повторной попыткой

# Запускаем бот в отдельном потоке при старте программы
bot_thread = threading.Thread(target=bot_thread_func, daemon=True)
bot_thread.start()

# --- Основная функция парсинга ---
def start_parsing(resume_from: Optional[str] = None) -> Dict[str, int]:
    """
    Главная функция парсинга с полным циклом обработки данных.
    
    Args:
        resume_from: Точка возобновления после сбоя
        
    Returns:
        Словарь с результатами: {
            'total_articles': int,
            'unique_articles': int,
            'spreadsheet_url': str
        }
    """
    global bot_running
    
    # Конфигурация (лучше вынести в отдельный config.py)
    # config = {
    #     'chat_id': "1938719365",
    #     'host': "https://vc.ru",
    #     'url': "https://vc.ru/new",
    #     'spreadsheet_id': "1XQBizX0YVDsZ7PaHlNvnEVLwyOxUZuFJo0447ku6Qdg",
    #     'max_scrolls': 5,
    #     'max_retries': 3,
    #     'retry_delay': 5
    # }
    
    result = {
        'total_articles': 0,
        'unique_articles': 0,
        'spreadsheet_url': f"https://docs.google.com/spreadsheets/d/{config.spreadsheet_id}"
    }

    try:
        # --- Этап 1: Инициализация ---
        if not resume_from:
            send_status_update(config.chat_id, "start", "Запуск парсера VC.ru")
            
            # Проверка незавершенных задач (реализацию load_failure_state нужно добавить)
            last_state = load_failure_state()
            if last_state:
                msg = f"Обнаружен незавершенный парсинг (этап: {last_state['resume_from']}). Продолжить?"
                if ask_confirmation(config.chat_id, msg):
                    resume_from = last_state['resume_from']
                    articles_data = last_state['data']

        # --- Этап 2: Динамический парсинг ---
        if not resume_from or resume_from == 'dynamic_parse':
            send_status_update(config.chat_id, "dynamic_parse", "Загрузка страницы...")
            
            html = dynamic_parse(
                url=config.url,
                max_scrolls=config.max_scrolls,
                max_retries=config.max_retries,
                retry_delay=config.retry_delay
            )
            
            if not html:
                raise Exception("Не удалось загрузить HTML")
                
            with open("../data/interim/dynamic_page.html", "w", encoding="utf-8") as f:
                f.write(html)
                
            send_status_update(config.chat_id, "dynamic_parse", "✅ HTML успешно сохранен")
            resume_from = None

        # --- Остальные этапы остаются без изменений ---
        # --- Этап 3: Парсинг статей ---
        if not resume_from or resume_from == 'parse_articles':
            send_status_update(config.chat_id, "parse_articles", "Обработка новостей...")
            
            parser = VcRuParser(host=config.host)
            articles_data = parser.parse_articles(html)
            result['total_articles'] = len(articles_data)
            
            send_status_update(config.chat_id, "parse_articles", f"✅ Найдено статей: {result['total_articles']}")
            resume_from = None

        # --- Этап 4: Обновление данных ---
        if not resume_from or resume_from == 'update_data':
            send_status_update(config.chat_id, "update_data", "Сохранение данных...")
            
            update_data(articles_data)
            send_status_update(config.chat_id, "update_data", "✅ Данные обновлены")
            resume_from = None

        # --- Этап 5: Экспорт в Google Sheets ---
        if not resume_from or resume_from == 'export_to_google_sheets':
            send_status_update(config.chat_id, "export", "Экспорт в Google Sheets...")
            
            result['unique_articles'] = export_to_google_sheets(
                data_source=articles_data,
                spreadsheet_id=config.spreadsheet_id,
                sheet_name="VC.ru News"
            )
            
            send_status_update(
                config.chat_id,
                "export",
                f"✅ Количество записей:\n\tПеред экспортом: {result['unique_articles'][0]}\n\tНовых: {result['unique_articles'][2]}\n\tСтало: {result['unique_articles'][1]}"
            )
            resume_from = None

        # --- Этап 6: Финиш ---
        if not resume_from:
            send_status_update(config.chat_id, "finish", "Парсер успешно завершён!")

        return result

    except Exception as e:
        logging.error(f"Critical error: {e}", exc_info=True)
        send_telegram_message(
            config.chat_id,
            f"⚠️ Критическая ошибка: {str(e)}",
            parse_mode="Markdown"
        )
        raise

    finally:
        # Корректное завершение работы бота
        bot_running = False
        try:
            bot.stop_polling()
        except:
            pass
        time.sleep(1)  # Даем время для завершения отправки

# --- Вспомогательные функции ---
def send_status_update(chat_id: str, step: str, message: str):
    """Отправка статусного сообщения с форматированием"""
    step_icons = {
        'start': "🚀",
        'dynamic_parse': "🔍",
        'parse_articles': "📝",
        'update_data': "💾",
        'export': "📤",
        'finish': "😊"
    }
    icon = step_icons.get(step, "ℹ️")
    text = f"{icon} {message}\n\t{datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"
    send_telegram_message(chat_id, text)

def ask_confirmation(chat_id: str, question: str) -> bool:
    """Запрашивает подтверждение через Telegram"""
    markup = types.InlineKeyboardMarkup()
    markup.add(
        types.InlineKeyboardButton("✅ Да", callback_data="confirm_yes"),
        types.InlineKeyboardButton("❌ Нет", callback_data="confirm_no")
    )
    
    msg = bot.send_message(chat_id, question, reply_markup=markup)
    
    # Ожидание ответа (упрощенная реализация)
    @bot.callback_query_handler(func=lambda call: call.message.message_id == msg.message_id)
    def handle_confirmation(call):
        if call.data == "confirm_yes":
            return True
        return False

@bot.callback_query_handler(func=lambda call: True)
def handle_callback(call):
    """Обработчик inline-кнопок"""
    try:
        if call.data.startswith("retry"):
            _, resume_point = call.data.split(":")
            send_telegram_message(call.message.chat.id, f"🔄 Перезапускаю с этапа: {resume_point}")
            start_parsing(resume_from=resume_point)
        elif call.data == "cancel":
            send_telegram_message(call.message.chat.id, "❌ Парсинг отменён")
    except Exception as e:
        logging.error(f"Callback error: {e}")

### 5. Пример использования - START_11 !!!
#### 5.1. Ручной запуск

In [12]:
#  - START_11 !!!
if __name__ == "__main__":
    try:
        start_parsing()
    except Exception as e:
        logging.critical(f"Fatal error: {e}")
    finally:
        # Дополнительная пауза перед завершением
        time.sleep(2)

2025-05-17 06:28:34,290 - WDM - INFO - ====== WebDriver manager ======
2025-05-17 06:29:32,592 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-05-17 06:29:33,131 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-05-17 06:29:33,444 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-05-17 06:29:34,270 - WDM - INFO - WebDriver version 136.0.7103.94 selected
2025-05-17 06:29:34,292 - WDM - INFO - Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.94/win32/chromedriver-win32.zip
2025-05-17 06:29:34,333 - WDM - INFO - About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.94/win32/chromedriver-win32.zip
2025-05-17 06:29:34,616 - WDM - INFO - Driver downloading response is 200
2025-05-17 06:29:41,397 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-05-17 06:29:51,476 - WDM - INFO - Driver has been saved in cache [C:\Users\User\

✅ 2. Новые данные сохранёны в переменной ARTICLES для обработки!
	Скрипт отработал: 17.05.2025 06:31:38


2025-05-17 06:31:41,235 - __main__ - INFO - Данные обновлены. Уникальных записей: 314. Новых записей: 1


✅ 3. CSV/JSON файлы сохранёны и дополнены новым контентом!
	Скрипт отработал: 17.05.2025 06:31:41


2025-05-17 06:31:50,727 - root - INFO - Старых записей: 313. Новых записей: 1. Экспортировано записей: 314.


✅ 3. Google Sheets дополнена новым контентом!
	Скрипт отработал: 17.05.2025 06:31:50
